<a href="https://colab.research.google.com/github/Twenkid/GPT2-Bulgarian-Training-Tips-and-Tools/blob/main/UPDATE-6-5-2023-TRAIN_NEW_GPT2_MEDIUM_BIG_DATASET_20_6_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training GPT2-MEDIUM from scratch on Colab
#UPDATE: 6-5-2023: float32 fix (no mixed precision - error) and
#Chained-generation with unlimited length with directed prompt-injection (see the other video from Todor's youtube channel)

Author: Todor Arnaudov - Twenkid, ~ 6/2021 - 9/2021
License: MIT

See a video tutorial: https://youtu.be/F-Xt-cK4L-g

GPT2-Medium Training from Scratch on Colab for Any Language - Tips & Tricks by Twenkid
...

Съвети за машинно обучение на GPT2-Medium модел на български или друг език от нулата през Google Colaboratory, от Тодор Арнаудов - Тош/Twenkid. Следва продължение.

The code was based on a tutorial for training locally the SMALL model by Arshabhi Kayal: Thanks!
https://towardsdatascience.com/train-gpt-2-in-your-own-language-fc6ad4d60171 

However it was extended with the proper parametrs, for loading data/saving back to Google drive, for randomly selecting part of the dataset in order to allow finishing iterations within the allowed Colab time etc. His example was for local training on an Nvidia Geforce RTX 2060 (6 GB) for GPT2-Small (3 times smaller) and a fixed dataset.

The parameters here are for GPT2-MEDIUM and run only with Tesla T4 16GB (or another future card with 16 GB or more).

The code in the experiments in this video was extended and debugged for application in the Colaboratory which has its hurdles and for gradual extension of the dataset after each training epoch, without retraining the tokenizer after the corpus is updated (see Dataset Skip None in the video).

I trained on a custom-built Bulgarian dataset. 

The code needs cleaning and more explanations - consult with the video for now.



I'm open for collaboration on interesting projects in various R&D domains, AGI and AI etc.

http://artificial-mind.blogspot.com
http://github.com/twenkid
http://research.twenkid.com

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from datetime import datetime
!pip install gensim
!pip install argparse
!pip install tokenizers
!pip install pathlib
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!nvidia-smi

Fri May  5 20:16:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# TRY mixed-precision 16bit float

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision


In [5]:
policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy('mixed_float16')
mixed_precision.set_global_policy('float32')

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [ ]:
!du -sh /content/ab

du: cannot access '/content/ab': No such file or directory


In [ ]:
!cp -r '/content/drive/MyDrive/corpus/' /content/ab/

#**Initial run for configuring the model and creating a tokenizer for the initial dataset.**

### **WARNING**: Don't run if you have already saved a tokenizer and a model. Run only the first time in order to create a tokenizer and configuration. Then after the first training save the model and afterwards just load it. Tokenize the corpus (you may have extended it in the meantime), select parts of it based on different ctireria if it is too big; create the dataset for tf and filter None items; compile the model and train for another epoch. Then save it etc.

In [ ]:
#DONT RUN IF NOT ALREADY SAVED TOKENIZER ETC.!
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

from pathlib import Path

#paths = [str(x) for x in Path("/content/drive/MyDrive/corpus/").glob("*.txt")]  
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")]  #Or edit accordingly to a Drive folder etc.
print(paths)
class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFKC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        print(paths)
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        #self.tokenizer.train(trainer, paths)
        self.tokenizer.train(paths,trainer)

    def bpe_train_2(self, paths):
        print(paths)
        #trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        trainer = BpeTrainer(vocab_size=2000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        #self.tokenizer.train(trainer, paths)
        self.tokenizer.train(paths,trainer)

    def bpe_train_3(self, paths):
        print(paths)
        #trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        #trainer = BpeTrainer(vocab_size=2000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        #self.tokenizer.train(trainer, paths)
        trainer = BpeTrainer(show_progress=True, inital_alphabet=ByteLevel.alphabet())
        self.tokenizer.train(paths,trainer)
    def bpe_train_4(self, paths):
        print(paths)
        #trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        #trainer = BpeTrainer(vocab_size=2000, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        #self.tokenizer.train(trainer, paths)
        trainer = BpeTrainer(vocab_size=50250, show_progress=True, inital_alphabet=ByteLevel.alphabet())
        self.tokenizer.train(paths,trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

#from tokenise import BPE_token
from pathlib import Path
import os
# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./text/").glob("**/*.txt")]
#paths = [str(x) for x in Path("./en/").glob("**/*.txt")]
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")]  #In one file for all, otherwise errors with the shapes?
#paths = [str(x) for x in Path("/content/ab/UTF8/").glob("*.txt")] 


#paths = [str(x) for x in Path("/content/ab2/corpus2/UTF8/").glob("*.txt")]

print(paths)
'''
for i in paths:
  print(i)
  f = open(i, "wt", encoding='utf-8')
  r = f.read()
  f.write(i,r)
'''  
tokenizerBIG = BPE_token()
# train the tokenizer model
tokenizerBIG.bpe_train_4(paths) #no ###########
# saving the tokenized data in our specified folder 
save_path = 'tokenized_data'
tokenizerBIG.save_tokenizer(save_path)

['/content/ab/radost.txt', '/content/ab/kakvomu_contents.txt', '/content/ab/ivo_hristov_svruh-obobshteniya_6-2021.txt', '/content/ab/telefonni-suveti.txt', '/content/ab/pred4.txt', '/content/ab/Ljuben_Dilov_-_Mnogoto_imena_na_straha_-_3560-b.txt', '/content/ab/RISC-CISC.txt', '/content/ab/Angel_Karalijchev_-_Bylgarski_narodni_prikazki_-_652-b.txt', '/content/ab/Ada_12-10_E-potursij-i-po-novi-vtoro-chislo-19-7-2020-Arial-p189_Arial16-Big-Font.txt', '/content/ab/democratius_prilozhenie_drugi.txt', '/content/ab/Isaac-Asimov_-_Fondatsija_i_Zemja_-_1025-b.txt', '/content/ab/comment-nerds-2-nerds-128-clean1.txt', '/content/ab/John-Braine_-_Pyt_kym_vissheto_obshtestvo_-_4406-b.txt', '/content/ab/zaklyuchena-poetry.txt', '/content/ab/Ivan Vazov - Tc - 3. Nova zemja - 3754.txt', '/content/ab/upaduk_dzbe.txt', '/content/ab/darenie.txt', '/content/ab/Isaac-Asimov_-_Fondatsija_i_Imperija_-_1022-b.txt', '/content/ab/Isaac-Asimov_-_Fondatsijata_-_1021-b.txt', '/content/ab/Gore-Vidal_-_Sytvorenieto_-

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving computers-Linus-gt-1.txt to computers-Linus-gt-1.txt
Saving computers-Linus-gtplus-2.txt to computers-Linus-gtplus-2.txt


In [6]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs mmio_stale_data retbleed
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtua

In [ ]:
!du -sh ab

3.0M	ab


In [ ]:
!cp -r '/content/drive/MyDrive/tokenized_data'  /content/


In [ ]:
#Or load directly
!cp -r '/content/drive/MyDrive/gpt2-medium'  /1024/

In [ ]:
!cp -r /content/ab/UTF8/* '/content/drive/MyDrive/corpus/'

In [ ]:
!cp -r /content/ab/* '/content/drive/MyDrive/corpus/'

In [ ]:
#COPY TOKENIZED DATA TO DRIVE

!cp -r /content/tokenized_data/* '/content/drive/MyDrive/tokenized_data/'


In [ ]:
#AFTER the model is saved

!cp -r /content/1024/* '/content/drive/MyDrive/1024/'

In [ ]:
#if needed 

from google.colab import files
files.upload()

`#**Create a single-string and tokenize (encode) the raw text**`

In [ ]:
tokenized = True
#IF NOT TOKENIZED!
'''
single_string = ''
tokenizerBIG.add_special_tokens({ #should add in text when feeding?!
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
  })
'''
items = []
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:    
    x = f.read()
    items.append(x)    
    #print(x)
  #single_string += x + tokenizerBIG.eos_token
  items.append(tokenizerBIG.eos_token)
single_string = ''.join(items)
print(len(single_string))

if (len(single_string)<999): print("\n===============\n"+single_string)
string_tokenized = tokenizerBIG.encode(single_string) #, truncation=True) #, max_length=100)
if (len(string_tokenized)<200000): print(string_tokenized)
print(len(string_tokenized))


NameError: ignored

# **Create the model when running for the first time**

In [ ]:
# CONFIG - BIG MODEL  BIG ...
# BIG BIG BIG
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
#from str import join

save_path = "/tokenized_data/"
# loading tokenizer from the saved model path
tokenizerBIG = GPT2Tokenizer(vocab_file='/content/tokenized_data/vocab.json', merges_file='/content/tokenized_data/merges.txt')
#Will it start now?  !!! merges.txt --> I used vocab.json!!! error bec... ?
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#SLOW?

from pathlib import Path
import os
# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./text/").glob("**/*.txt")]
#paths = [str(x) for x in Path("./en/").glob("**/*.txt")]
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")]  #In one file for all, otherwise errors with the shapes?
paths = [str(x) for x in Path("/content/ab2/").glob("*.txt")] 
#paths = [str(x) for x in Path("/content/ab2/corpus2/UTF8/").glob("*.txt")]

print(paths)


'''
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
'''

loaded = False #True
if (not loaded):
  tokenizerBIG.add_special_tokens({ #should add in text when feeding?!
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
  })

# creating the configurations from which the model can be made
configBIG = GPT2Config(
  vocab_size=tokenizerBIG.vocab_size,
  bos_token_id=tokenizerBIG.bos_token_id,
  eos_token_id=tokenizerBIG.eos_token_id,
  n_embd=1280,
  n_head = 20,
  n_layer = 20 #24 ... experiments
)

#~ MEDIUM
num_heads = 16
configBIG = GPT2Config(
  vocab_size=tokenizerBIG.vocab_size,
  bos_token_id=tokenizerBIG.bos_token_id,
  eos_token_id=tokenizerBIG.eos_token_id,
  n_embd= 1024,
  n_head = num_heads,
  n_layer = 24 #num_layers #24 #24
)

'''
num_heads = 18
configBIG = GPT2Config(
  vocab_size=tokenizerBIG.vocab_size,
  bos_token_id=tokenizerBIG.bos_token_id,
  eos_token_id=tokenizerBIG.eos_token_id,
  n_embd= 1152,
  n_head = num_heads,
  n_layer = 36 #num_layers #24 #24
)
'''
'''
n_embd=1280,
n_head = 20,
n_layer = 20 
block_size = 100BATCH_SIZE = 20
BUFFER_SIZE = 5000
BATCH_SIZE == n_layer  ? (otherwise can't fit, see shapes)
'''

# X OK -- fits 14218 MB - Large model -- no.
''' OK - fits also!
configBIG = GPT2Config(
  vocab_size=tokenizerBIG.vocab_size,
  bos_token_id=tokenizerBIG.bos_token_id,
  eos_token_id=tokenizerBIG.eos_token_id,
  n_embd=1600,  
)
'''
''' NO:
configBIG = GPT2Config(
  vocab_size=tokenizerBIG.vocab_size,
  bos_token_id=tokenizerBIG.bos_token_id,
  eos_token_id=tokenizerBIG.eos_token_id,
  n_embd=1600,
  n_layer=48,
  n_ctx = 2048,
  n_head = 12,
)
'''

configBIG.vocab_size+=5

print("Vocab size = "+ str(configBIG.vocab_size))

# creating the model
modelBIG = TFGPT2LMHeadModel(configBIG)

print("single_string-->")
single_string = ''
items = []
n = 1
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:    
    x = f.read()
    items.append(x) 
    print(n, filename)
    n+=1
    #print(x)
  #single_string += x + tokenizerBIG.eos_token
  items.append(tokenizerBIG.eos_token)
single_string = ''.join(items)
print(len(single_string))

if (len(single_string)<999): print("\n===============\n"+single_string)
string_tokenized = tokenizerBIG.encode(single_string) #, truncation=True) #, max_length=100)
if (len(string_tokenized)<200000): print(string_tokenized)
print(len(string_tokenized))

examples = []
#block_size = 100
#BATCH_SIZE = 12
#BUFFER_SIZE = 1000
#block_size = 32
#BATCH_SIZE = 4
#BUFFER_SIZE = 64
#... msg BatchDataset: dataset BatchDataset with 1101 items ?
# shape [10,99],  [2, 10, 12, 99]  10,12 should be equal? ... 1101919 tokens
#if batch_size = 100--> [100,99], [2,100, 12, 99] - last is block_size-1?
#!! OK: with: block_size=100; batch_size=12; BUFFER_SIZE=2000

block_size = 100 #100 #256 - resources exhausted, block 256, bs=20 #100
BATCH_SIZE = num_heads  # # 20 # #24 #for 20 layers - model 1280,36,20; 12 for small model 768 ... #12
BUFFER_SIZE = 3000 #  4000 - interrupts connection after one minute; 5000 #3072 #9216 #1000

#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
smaller = string_tokenized[0:len(string_tokenized)//3]
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
#  examples.append(string_tokenized[i:i + block_size])
  smaller = string_tokenized[0:len(string_tokenized)/3]
inputs, labels = [], []
k = 0
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  if k==0:
    print(inputs[:100])
    print(labels[:100])
  k+=1
  #print(inputs)
  #print(labels)
print(k)
print(len(inputs))
print(len(labels))
i1 = len(inputs); i2 = len(labels)
i3 = i1*BATCH_SIZE
i4 = i1*block_size
print(i3)
print(i4)
datasetBIG = tf.data.Dataset.from_tensor_slices((inputs, labels))
datasetBIG = datasetBIG.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(len(string_tokenized))

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
modelBIG.compile(optimizer=optimizer, loss=[loss, *[None] * modelBIG.config.n_layer], metrics=[metric])
print("MODEL COMPILED")

#PICKLE!

import pickle
f = open("string_tokenized.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(string_tokenized)  # pickle.HIGHEST_PROTOCOL
f.close()
f2 = open("dataset.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(dataset)  # pickle.HIGHEST_PROTOCOL
f.close()
print("dataset = ", dataset)

!cp -r /content/dataset.pickle '/content/drive/MyDrive/dataset.pickle
print("Copied Dataset to drive")

num_epoch = 1
historyBIG = modelBIG.fit(datasetBIG, epochs=num_epoch)

['/content/ab/Krystxo_Manchev_-_Istorija_na_balkanskite_narodi._Tom_1_-_1352-1878_-_1962-b.txt', '/content/ab/necessary_illusions.txt', '/content/ab/Hristo Botev - Vnbe - . Narodyt - vchera, dnes i utre - 3636.txt', '/content/ab/Hristo Botev - V - . Edinstvenoto spasenie na nashija narod e v revoljutsijata - 3644.txt', '/content/ab/Anglijski_narodni_prikazki_-_3637-b.txt', '/content/ab/Hristo Botev - V - . Kakvo mozhe da napravi nasheto duhovenstvo - 3697.txt', '/content/ab/Zapadnoevropejski_poeti_romantitsi_-_Antologija_-_262-b.txt', '/content/ab/Emilija_Drumeva_-_Konstitutsionno_pravo_-_5956-b.txt', '/content/ab/Platon_-_Dyrzhavata_-_9825-b.txt', '/content/ab/Hristo Botev - V - . Shte vidite - 3687.txt', '/content/ab/democratius_prilozhenie_drugi.txt', '/content/ab/EIM-smetach-do-br-6-moje-da-e-nepulno-bg.txt', '/content/ab/telefonni-suveti.txt', '/content/ab/sreshtu-chuzhdico.txt', '/content/ab/Karl-May_-_Vinetu_II_-_1326-b.txt', '/content/ab/Hristo Botev - V - . Zadachata na v. ,,Z

TypeError: ignored

In [ ]:
#PICKLE!

import pickle
f = open("string_tokenized.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(string_tokenized)  # pickle.HIGHEST_PROTOCOL
f.close()
#f2 = open("dataset.pickle", "wb")
#pickle.Pickler(f,protocol=4).dump(dataset)  # pickle.HIGHEST_PROTOCOL
#f.close()

In [ ]:
from pathlib import Path
import os
# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./text/").glob("**/*.txt")]
#paths = [str(x) for x in Path("./en/").glob("**/*.txt")]
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")]  #In one file for all, otherwise errors with the shapes?
paths = [str(x) for x in Path("/content/ab2/").glob("*.txt")]


In [ ]:
from pathlib import Path
import os
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")] 

In [ ]:
#sdfsdofjsdoifj ERRROR SPRI!
#dfdfd
examples = []
#block_size = 100
#BATCH_SIZE = 12
#BUFFER_SIZE = 1000
#block_size = 32
#BATCH_SIZE = 4
#BUFFER_SIZE = 64
#... msg BatchDataset: dataset BatchDataset with 1101 items ?
# shape [10,99],  [2, 10, 12, 99]  10,12 should be equal? ... 1101919 tokens
#if batch_size = 100--> [100,99], [2,100, 12, 99] - last is block_size-1?
#!! OK: with: block_size=100; batch_size=12; BUFFER_SIZE=2000

block_size = 120 #100 #256 - resources exhausted, block 256, bs=20 #100
BATCH_SIZE = num_heads  # # 20 # #24 #for 20 layers - model 1280,36,20; 12 for small model 768 ... #12
BUFFER_SIZE = 3000 #  4000 - interrupts connection after one minute; 5000 #3072 #9216 #1000

#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
maxsize = int(len(string_tokenized)*0.8)
smaller = string_tokenized[0:len(string_tokenized)] #//2]
#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
#  examples.append(string_tokenized[i:i + block_size])
bReverse = True #try
if not bReverse:
  for i in range(0, len(smaller) - block_size + 1, block_size):
    examples.append(smaller[i:i + block_size])
else if bReverse: #Very long epochs, 3:35 h -- sometimes run in half, but reverse (it's randomized also later; rather generate the dataset again)
  for i in range(len(smaller) - block_size + 1, 0, -block_size):
    examples.append(smaller[i:i + block_size])

 
inputs, labels = [], []
k = 0
for ex in examples:
  inputs.append(ex[:-1]) #inefficient
  labels.append(ex[1:])
  if k==0:
    print(inputs[:100])
    print(labels[:100])
  k+=1
  if (k%1000==0): print("EX: "+str(k),end=' ')
  #print(inputs)
  #print(labels)
print(k)
print(len(inputs))
print(len(labels))
i1 = len(inputs); i2 = len(labels)
i3 = i1*BATCH_SIZE
i4 = i1*block_size
print(i3)
print(i4)
datasetBIG = tf.data.Dataset.from_tensor_slices((inputs, labels))
datasetBIG = datasetBIG.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(len(string_tokenized))

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
modelBIG.compile(optimizer=optimizer, loss=[loss, *[None] * modelBIG.config.n_layer], metrics=[metric])
print("MODEL COMPILED")

#PICKLE!

import pickle
f = open("string_tokenized.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(string_tokenized)  # pickle.HIGHEST_PROTOCOL
f.close()
#f2 = open("dataset.pickle", "wb")
#pickle.Pickler(f2,protocol=pickle.HIGHEST_PROTOCOL).dump(datasetBIG)  # pickle.HIGHEST_PROTOCOL
#f2.close()
#print("dataset = ", dataset)
#
#InternalError: Tensorflow type 21 not convertible to numpy dtype.

!cp -r /content/dataset.pickle '/content/drive/MyDrive/dataset.pickle
print("Copied Dataset to drive")

num_epoch = 1 #1
historyBIG = modelBIG.fit(datasetBIG, epochs=num_epoch)

[[16632, 140, 538, 4579, 1579, 8849, 3814, 141, 140, 32016, 211, 15605, 4326, 13, 3224, 497, 141, 140, 2288, 11968, 1057, 49899, 141, 141, 141, 140, 1470, 30890, 3463, 228, 9838, 224, 625, 20165, 222, 39387, 3327, 13, 26530, 546, 11, 273, 28464, 8499, 11, 4672, 695, 233, 2974, 9648, 11, 31283, 203, 4005, 29358, 198, 1508, 1761, 1007, 11, 239, 1864, 11, 584, 1616, 202, 3529, 211, 32515, 211, 923, 197, 2390, 13719, 202, 205, 44631, 211, 6853, 11, 2701, 198, 1289, 13, 836, 16905, 2663, 15153, 248, 13540, 345, 13, 2828, 700, 273, 546, 6988, 211, 612, 9678, 198, 1645, 1461, 3386, 228, 2297, 874, 850, 6073, 11, 3527, 799, 4323, 258, 13, 141, 140, 15040, 3960]]
[[140, 538, 4579, 1579, 8849, 3814, 141, 140, 32016, 211, 15605, 4326, 13, 3224, 497, 141, 140, 2288, 11968, 1057, 49899, 141, 141, 141, 140, 1470, 30890, 3463, 228, 9838, 224, 625, 20165, 222, 39387, 3327, 13, 26530, 546, 11, 273, 28464, 8499, 11, 4672, 695, 233, 2974, 9648, 11, 31283, 203, 4005, 29358, 198, 1508, 1761, 1007, 11, 239,

KeyboardInterrupt: ignored

In [ ]:
# AFTER LOADING

examples = []

block_size = 120 #100 #256 - resources exhausted, block 256, bs=20 #100
BATCH_SIZE = num_heads  # # 20 # #24 #for 20 layers - model 1280,36,20; 12 for small model 768 ... #12
BUFFER_SIZE = 3000 #  4000 - interrupts connection after one minute; 5000 #3072 #9216 #1000

#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
maxsize = int(len(string_tokenized)*0.8)
smaller = string_tokenized[0:len(string_tokenized)] #//2]
#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
#  examples.append(string_tokenized[i:i + block_size])
bReverse = True #try
if not bReverse:
  for i in range(0, len(smaller) - block_size + 1, block_size):
    examples.append(smaller[i:i + block_size])
else if bReverse: #Very long epochs, 3:35 h -- sometimes run in half, but reverse (it's randomized also later; rather generate the dataset again)
  for i in range(len(smaller) - block_size + 1, 0, -block_size):
    examples.append(smaller[i:i + block_size])

inputs, labels = [], []
k = 0
for ex in examples:
  inputs.append(ex[:-1]) #inefficient
  labels.append(ex[1:])
  if k==0:
    print(inputs[:100])
    print(labels[:100])
  k+=1
  if (k%1000==0): print("EX: "+str(k),end=' ')
  #print(inputs)
  #print(labels)
print(k)
print(len(inputs))
print(len(labels))
i1 = len(inputs); i2 = len(labels)
i3 = i1*BATCH_SIZE
i4 = i1*block_size
print(i3)
print(i4)
datasetBIG = tf.data.Dataset.from_tensor_slices((inputs, labels))
datasetBIG = datasetBIG.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(len(string_tokenized))

In [ ]:
num_epoch = 1 #1
historyBIG = modelBIG.fit(datasetBIG, epochs=num_epoch)

 749/8482 [=>............................] - ETA: 3:08:10 - loss: 5.3441 - logits_loss: 5.3441 - logits_accuracy: 0.2087 - past_key_values_1_accuracy: 0.0018 - past_key_values_2_accuracy: 0.0021 - past_key_values_3_accuracy: 0.0016 - past_key_values_4_accuracy: 0.0019 - past_key_values_5_accuracy: 0.0019 - past_key_values_6_accuracy: 0.0017 - past_key_values_7_accuracy: 0.0021 - past_key_values_8_accuracy: 0.0018 - past_key_values_9_accuracy: 0.0021 - past_key_values_10_accuracy: 0.0020 - past_key_values_11_accuracy: 0.0016 - past_key_values_12_accuracy: 0.0022 - past_key_values_13_accuracy: 0.0023 - past_key_values_14_accuracy: 0.0019 - past_key_values_15_accuracy: 0.0019 - past_key_values_16_accuracy: 0.0018 - past_key_values_17_accuracy: 0.0017 - past_key_values_18_accuracy: 0.0017 - past_key_values_19_accuracy: 0.0020 - past_key_values_20_accuracy: 0.0022 - past_key_values_21_accuracy: 0.0021 - past_key_values_22_accuracy: 0.0016 - past_key_values_23_accuracy: 0.0018 - past_key_val

In [ ]:
print(len(string_tokenized))

19752522


In [ ]:

num_epoch = 1
historyBIG = modelBIG.fit(datasetBIG, epochs=num_epoch)

4415/8482 [==============>...............] - ETA: 1:38:51 - loss: 5.8585 - logits_loss: 5.8585 - logits_accuracy: 0.1884 - past_key_values_1_accuracy: 0.0018 - past_key_values_2_accuracy: 0.0023 - past_key_values_3_accuracy: 0.0016 - past_key_values_4_accuracy: 0.0018 - past_key_values_5_accuracy: 0.0018 - past_key_values_6_accuracy: 0.0019 - past_key_values_7_accuracy: 0.0022 - past_key_values_8_accuracy: 0.0017 - past_key_values_9_accuracy: 0.0023 - past_key_values_10_accuracy: 0.0023 - past_key_values_11_accuracy: 0.0018 - past_key_values_12_accuracy: 0.0022 - past_key_values_13_accuracy: 0.0024 - past_key_values_14_accuracy: 0.0020 - past_key_values_15_accuracy: 0.0020 - past_key_values_16_accuracy: 0.0017 - past_key_values_17_accuracy: 0.0018 - past_key_values_18_accuracy: 0.0019 - past_key_values_19_accuracy: 0.0020 - past_key_values_20_accuracy: 0.0024 - past_key_values_21_accuracy: 0.0022 - past_key_values_22_accuracy: 0.0016 - past_key_values_23_accuracy: 0.0014 - past_key_val

KeyboardInterrupt: ignored

In [ ]:
# matplotlib interprets images in RGB format, but OpenCV uses BGR format
# so to convert the image so that it's properly loaded, convert it before loading
#TEST 20-6-2021
import cv2
import matplotlib as plt

img = cv2.imread('1.jpg')		# this is read in BGR format
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)		# this converts it into RGB
plt.imshow(rgb_img)
plt.show()

In [ ]:
print(len(string_tokenized))

21581913


In [ ]:
#PICKLE!

import pickle
f = open("stringg_tokenized.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(string_tokenized)  # pickle.HIGHEST_PROTOCOL
f.close()
f2 = open("dataset.pickle", "wb")
pickle.Pickler(f,protocol=4).dump(dataset)  # pickle.HIGHEST_PROTOCOL
f.close()
print("dataset = ", dataset)

!cp -r /content/dataset.pickle '/content/drive/MyDrive/dataset.pickle

infile = open(dataset.pickle,'rb')
dataset_load = pickle.load(infile)
infile.close()

#!cp -r /content/dataset.pickle '/content/drive/MyDrive/dataset.pickle'

print("dataset_load = ", dataset_load)
#dataset2 = pickle.

In [ ]:
# BIG GENERATE ...
# BBBB IIIII GGGGG 
text = "Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът Емил."""
text = "Имало едно време един принц, който имал златно сърце и се запознал с баба Яга. Тя била любовница на юнака и спяла със Снежанка и Терминатора, който програмирал Intel процесори."""
text = "Какви са минималните изисквания за играта? Попитал Тодор и се обадил на изкуствения интелект."
text = "Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът Емил."
#text = "Универсалният изкуствен разум и какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Демокрацията е мит, всичко е тоталитарна свобода на словото и любовта. Емил се усмихна на процесора."
text = "Не съм съгласен! Той отвори вратата и напусна стаята, в която бяха останали само димящата цигара на Мария и нейните сълзи."
text = "Не мога, скъпа! - Защо? - попита тя. - Ти си толкова красива, но искам да гледам мача довечера. Ще играят Ботев и Локомотив. - О, скъпи, няма проблем, тъкмо ще се видя с майка ми."
number = 1
# encoding the input text
input_ids = tokenizerBIG.encode(text, return_tensors='tf')
# getting out output
beam_output = modelBIG.generate(
  input_ids,
  max_length = 200, # 400, #80
  num_beams = 16, #12, 6 # 5,
  temperature = 1, #1, #1, #1.0, #1,
  top_k = 40, #, 40, #40
  #top_p = 0.95,
  no_repeat_ngram_size=2, #4, #2
  num_return_sequences=number
)
for i in range(0,number):
  print(tokenizerBIG.decode(beam_output[i]))

Setting `pad_token_id` to 50250 (first `eos_token_id`) to generate sequence


Не мога, скъпа! - Защо? - попита тя. - Ти си толкова красива, но искам да гледам мача довечера. Ще играят Ботев и Локомотив. - О, скъпи, няма проблем, тъкмо ще се видя с майка ми.

    Не знам, че не е по-малко от мен, за да не мога да се случи, защото не се е в състояние, а не може да си кажа какво е, и да е така?
	И така, ако не би могъл да го е случило, аз не ми се струва. Но аз, ти не знам какво да ме е вярно, ще трябва да ви кажа, та да ти кажа. И аз ще ти казвам, не ти се казвам аз съм го казвам. А аз да ми кажа аз. Ако не съм си казал какво ще ви казвам: „Какво ще ме казвам — казвам ти го ти, да те казвам?“ — отвърнал. Не мога и ще ми казвам какво ти е това, мисля, тя,


In [ ]:
!du -sh /content/ab/UTF8/

140M	/content/ab/UTF8/


In [ ]:
def ensure_utf(path = "."):
    #if path = None:
    #path = r'Z:/knigi/'
    import os
    import glob
    ls = os.listdir(path)
    print(ls)
    os.chdir(path)    
    print("========")
    print("Unzip")
    '''
    for i in ls:
      #cmd = "unzip + i + " " + "-d " + i+".txt"  #-q quite?  "-d " - directory specify
      #cmd = "unzip " + i  #+ i+".txt"  #-q quite?      
      #cmd = "unzip -l " + "\"" + i + "\""
      cmd = "unzip -j " + "\"" + i + "\""  #without creating dir
      print(cmd)
      os.system(cmd)
    '''
    ls = glob.glob("*.txt")
    print(ls)
    try:
      os.mkdir('UTF8')
    except:
           print("Dir already exists")
    for i in ls:
      print("TO UTF8:", i)
      f = open(i, "rt", encoding='utf-8')
      r = f.read()
      f2 = open(path+"/UTF8/"+i, "wt", encoding='utf-8')
      f2.write(r)
      f.close()
      f2.close()

In [ ]:
ensure_utf('/content/ab/')

['Ljubomir_Nikolov_-_Konan_i_igrata_na_bogovete_-_5334-b.txt', 'ivo_hristov_svruh-obobshteniya_6-2021.txt', 'alyosha2.txt', 'Lev_Tolstoj_-_Ana_Karenina_-_3066-b.txt.zip', 'Vladimir_Nabokov_-_Lolita_-_275-b.txt.zip', 'yunaka-i-taz.txt', 'Ernest-Hemingway_-_Za_kogo_bie_kambanata_-_5942-b.txt.zip', 'tupanasov.txt', 'Sun_Dzy_-_Izkustvoto_na_vojnata_-_1696-b.txt.zip', 'Giacomo-Casanova_-_Prikljuchenijata_na_Kazanova_-_Tom_1_-_6751-b.txt', 'EIM-smetach-do-br-6-moje-da-e-nepulno-bg.txt', 'terminator-i-herkules.txt', 'John-Steinbeck_-_Na_iztok_ot_Raja_-_1953-b.txt.zip', 'Geoffrey-Chaucer_-_Kentyrbyrijski_razkazi_-_4125-b.txt.zip', 'Hiljada_i_edna_nosht_-_Tom_II_-_6538-b.txt.zip', 'Ivan_Hadzhijski_-_Optimistichna_teorija_za_nashija_narod_-_7943-b.txt.zip', 'Charles-Perrault_-_Chervenata_shapchitsa_i_drugi_prikazki_-_10184-b.txt', 'kakvomu_notes.txt', 'Emilija_Drumeva_-_Konstitutsionno_pravo_-_5956-b.txt', 'Brian-Sibley-John-Howe_-_Karta_na_Tolkinovata_Sredna_zemja_-_6310-b.txt', 'talev-svetilni

In [ ]:
ensure_utf('/content/sample_data/ab/')

['Hristo Botev - Vnbe - . Namesto programa - 3635.txt', 'Ivan_Vazov_-_Razkazi_1881-1901_-_508-b.txt', 'George-Orwell_-_1984_-_53-b.txt', 'Michael-H-Hart_-_100-te_naj-vlijatelni_lichnosti_v_istorijata_na_chovechestvoto_-_1652-b.txt', 'Emilija_Drumeva_-_Konstitutsionno_pravo_-_5956-b.txt', 'igrachi-yunaci.txt', 'igrach-mekichar.txt', 'Hristo Botev - V - . Zadachata na v. ,,Zname_ - 3641.txt', 'Hristo Botev - V - . Podslushano - 3675.txt', 'Giovanni-Boccaccio_-_Dekameron_-_29-b.txt', 'Hristo Botev - V - . Vazhno neshto - 3681.txt', 'Pavel_Vezhinov_-_Vezni_-_10103-b.txt', 'bivalica.txt', 'Hristo Botev - Vnbe - . ,,Jugoslavija_ - 3640.txt', 'podvizi-akro-mezo-stih.txt', 'Terry-Pratchett_-_Dzhoni_i_bombata_-_4211-b.txt', 'Erich-Kastner_-_Kogato_bjah_malko_momche_-_10252-b.txt', 'Isaac-Asimov_-_Ostrieto_na_Fondatsijata_-_1024-b.txt', 'Nikola_M._Nikolov_-_Svetovnata_konspiratsija_-_Tova_koeto_istoritsite_ne_vi_kazvat_-_3694-b.txt', 'Hristo Botev - V - . Obrezanie na zhenite - 3683.txt', 'Sredn

In [ ]:
!ls /content/ab/
!du -sh /content/ab/

'_-  - 2. Biblija - Nov zavet - 2256.txt'
 Afrikanski_prikazki_-_Prerazkazani_ot_Renata_Pakarie_-_651-b.txt
 Agatha-Christie_-_Deset_malki_negyrcheta_-_1839-b.txt
 Alan-Williams_-_Arhivyt_na_Berija_-_Kniga_pyrva_-_10179-b.txt
 Aldous-Huxley_-_Prekrasnijat_nov_svjat_-_625-b.txt
 Aleko_Konstantinov_-_Baj_Ganxo_-_1763-b.txt
'Aleko Konstantinov -  - . Do Chikago i nazad - 3705.txt'
 Aleko-Schweizariya.txt
 Alexandre-Jardin_-_Zebrata_-_6966-b.txt
 Alex-Bradley_-_24_momicheta_za_7_dni_-_6383-b.txt
 Andre-Maurois_-_Bajron_-_3615-b.txt
 Angel_Karalijchev_-_Bylgarski_narodni_prikazki_-_652-b.txt
 Anglijski_narodni_prikazki_-_3637-b.txt
 Anthony-Capella_-_Ako_iskash_da_si_legnesh_s_men..._sgotvi_mi_-_5658-b.txt
 Arthur-Hailey_-_Letishte_-_32-b.txt
 Arthur-Hailey_-_Vechernite_novini_-_2402-b.txt
 Balzak-chast.txt
 Benjamin-Spock_-_Grizhi_za_bebeto_i_deteto_-_1475-b.txt
 Biblija_-_1435-b.txt
 Boris_Kitanov_-_Razpoznavane_i_sybirane_na_bilki_-_1453-b.txt
 Bruder-Grimm_-_Prikazki_-_648-b.txt
 Bylgar

In [ ]:
text = "Компютрите са бързи, но хората обичат да летят покрай космически кораби."
text = "Членът отишъл в космическия кораб и излетял."
text = "What is Todor?"
#text = "Ада отвори вратата и Емил я последва. На земята лежаха златни бижута."
text = "Ада отвори вратата и Емил я последва. Коридорът светеше до бижутера, когато Тичко се уплаши. Чернокосата дръпна метлата."
text = "Чернокосата ритна Тичко в топките и той излетя с метлата. Ана го погледна и се разсмя, а Емил падна с лазера."
text = "Обичам книги и Ана. Асансьорът потегли и срещнаха Широкоплещестият обаче метлата излетя от коридора и падна при картините в асансьора."
#text = "Ана, ти си единственото заради което момче като мен има сили да живее."
#text = "Компютрите са бързи, но хората обичат да летят покрай космически кораби."
#Ада отвори вратата и Емил я последва в Англия. Коридорът светеше до бижутера, когато той се страхуваше за свободата си. Какво му трябва на човек
text = "Най-бързият процесор е 1 ГХц и има 500 милиона транзистора, но не е по-добър от Жигули, най-хубавата кола според песните на Наско Ментата. Какво му трябва на човек, освен едно ферари?"
#text = """Да бъдеш или да не бъдеш - попита проститутката? Това е въпросът, задник! - отговори Терминаторът - застреля БМВ-то с пушката си и колата с Джон Конър се преобърна. Тогава момичето засмука хия му, докато той не свърши в устата и тя рече: О, толкова ти е голям!"""
text = """Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът - застреля президента с пушката си и колата с Емил се преобърна. Тогава момичето го целуна и те умряха."""
text = """Програмирането и изкуствения интелект са тъпотии, с които не се занимават феминистки. Демокрацията е мит."""
text = """Универсалният изкуствен разум и какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Демокрацията е мит, всичко е тоталитарна свобода на словото и любовта. Емил се усмихна на процесора."""
text = """Какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Рап. Демокрацията е мит. Любовта и свещеният сметач са най-добрите - рече Тош и се обади на държавата."""
text = """Какво му трябва на човек? Парите, колите, жените и песните!"""
text = """Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът - застреля президента с пушката си и колата с Емил се преобърна. Тогава момичето го целуна и те умряха."""
text = "Членът на ЦК на БКП отишъл в космическия кораб и излетял при лордовете господари от Англия, когато нацистите кацнали в България със сметачите си Правец-8М и се видели с Ангел и Тош пред телевизора. Тогава Фейз се обадил на Коко и отишли да ритат пред блока."
text = "Най-бързият процесор е 1 ГХц и има 500 милиона транзистора, но не е по-добър от Жигули, най-хубавата кола според песните на Наско Ментата. Какво му трябва на човек, освен едно ферари?"
text = "Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът Емил."""
number = 1
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 200, # 400, #80
  num_beams = 12, #12, 6 # 5,
  temperature = 1, #1.0, #1,
  top_k = 10, #, 40, #40
  top_p = 0.95,
  no_repeat_ngram_size=2, #4, #2
  num_return_sequences=number
)
for i in range(0,number):
  print(tokenizer.decode(beam_output[i]))

In [ ]:
#SAVE BIG   
   
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = '/content/1024/'
output_dir = '/content/gpt2/' #FOR tosh.bg ...
output_dir = '/content/drive/MyDrive/gpt2-medium/'
#or drive
#output_dir = '/content/drive/My Drive/gpt-bg/'  #for emil

# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = modelBIG.module if hasattr(modelBIG, 'module') else modelBIG
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
modelBIG.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizerBIG.save_pretrained(output_dir)

# 1.31 GB model for
'''
~ MEDIUM 
 n_embd= 1024,
 n_head = 16
 n_layer = 24 
'''
'''
The default small model: 474 MB
'''

'\nThe default small model: 474 MB\n'

In [ ]:
#SAVE SMALL

from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = './model_bn_custom/'
#or drive
output_dir = '/content/drive/My Drive/gpt-bg/'

# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

# **LOAD MODEL**

## Run after you have a saved model in your Drive.

In [7]:
#LOADMODEL  adjust which tokenizer etc.
#SAVE ALSO THE DATASET! PICKLE - Can't pickle it, incompatible
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer

load_dir = "/content/1024/"
load_dir = '/content/drive/MyDrive/gpt2/' #@ TWENKID
tokenizer2 = GPT2Tokenizer.from_pretrained(load_dir)
model2 = TFGPT2LMHeadModel.from_pretrained(load_dir)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/gpt2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
!ls /content/ab/

In [13]:
model2

In [9]:
tokenizer2 = GPT2Tokenizer.from_pretrained(load_dir)

In [ ]:
ls '/content/drive/MyDrive/gpt2-medium/'

ls: cannot access '/content/drive/MyDrive/gpt2-medium/': No such file or directory


In [ ]:
from pathlib import Path
import os
import random

# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./text/").glob("**/*.txt")]
#paths = [str(x) for x in Path("./en/").glob("**/*.txt")]
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/UTF8/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/2/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/sample_data/ab/UTF8/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")] 
print(len(paths))

319


# **TOKENIZE - ТОКЕНИЗЕ**


In [ ]:
#TOKENIZE - Only part of the Dataset for now - after loading AFTER LOADING  ... tokenizer2
#set devider to allow for faster epoch iterations
divider = 1 #2 #1 #4 #1 # #2 #1 #5, 3 
shifter = 0 #0 #0  #start not from the first
iReverse = 0 #1 #0 #1 # 1 # reverse the dir list
windowmax = 150000
location = 0.0 #beginning, 1.0 - ending
iFromTheEnd = 0
iPaddingEnd = 0 #500 # 1500 #skip final notes, book signature etc.

windowmax = 200000 # 100000 #120000 #80000 #40000 #50000 #100000 #100000 #smaller window, reverse
#windowmax = 100000 #from 10.7.2021 --> more iterations at random places
#windowmax = 100000 #22.7.2021
#windowmax = 200000 
#windowmax = 100000 
windowmax = 200000 #75000 
iReverse = 0 #1 #0 #1 #0 #1
bRandomLocation = False #True 
startPos = 0
start = 0
bShiftOneStep = True #start from windowmax # 4.7.2021

from pathlib import Path
import os
import random

# the folder 'text' contains all the files
#paths = [str(x) for x in Path("./text/").glob("**/*.txt")]
#paths = [str(x) for x in Path("./en/").glob("**/*.txt")]
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/UTF8/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/2/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/sample_data/ab/UTF8/").glob("*.txt")] 
paths = [str(x) for x in Path("/content/ab/").glob("*.txt")] 
#paths = [str(x) for x in Path("/content/a").glob("*.txt")]  #check one by one

#ADD big twice: for second sampling, #22-7-2021
additional = []
for filename in paths:
   with open(filename, "r", encoding='utf-8') as f:   
     x = f.read()
     #if (len(x)>windowmax*8): #if (len(x)>windowmax):
     #if (len(x)>windowmax*1.1) and (len(x)<windowmax*6): # and (len(x)<windowmax*1): #if (len(x)>windowmax):
     #if ((len(x)>windowmax*1.1) and (len(x)<windowmax*6)) or (len(x)>1100000): # and (len(x)<windowmax*1): #if (len(x)>windowmax):
     if ((len(x)>windowmax*1.5)): # and (len(x)<windowmax*6)) or (len(x)>1100000): # and (len(x)<windowmax*1): #if (len(x)>windowmax):
        additional.append(filename)

defLen = len(paths)
paths += additional

print(paths)
selected_paths = []
items = []
k = shifter # 0 - 1.0
pathslen = len(paths)
#if reverse...
if iReverse!=0: paths.reverse()
maxfiles = 500*divider #change >
top = 0
files = {}
step = 0
#for filename in paths: 
for filename in paths: 
  if k >= pathslen: break
  if k > maxfiles: break
  with open(filename, "r", encoding='utf-8') as f:        
    if k%divider != 0: k+=1; print("CONTINUE",k); continue #1/divider
    #if k%divider >1: continue #1/divider
    x = f.read()
    if (len(x) > windowmax):
        #if len(x)>=2*windowmax: #4.7.2021
        if len(x)>=windowmax: #6.7.2021
          if bShiftOneStep:
            rat = windowmax/len(x)
            if (rat < 0): rat = 0.0            
            #fl = max(random.random(),rat) #- rat) #, len(x)-windowmax) #>0.0 0.9 # - len(x)*0.1            
            #start = windowmax + fl*len(x)
            #start = windowmax//2 + fl*len(x) 5.7
            fl = random.random()
            if (fl > (1.0 - rat)): fl = 1.0-rat #to fill the windowmax - check the debug output
            start = fl*len(x) #6.7
            print("bShiftOneStep!")
            print("start=",start)
          else: start = startPos; print("start(default)=",start)  #default 0           
        if bRandomLocation:
          fl = random.random()
          start = int(fl)*len(x)
          items.append(x[start:min(start+windowmax-iPaddingEnd, len(x)-iPaddingEnd)])
          step = min(start+windowmax-iPaddingEnd, len(x)-iPaddingEnd - start)
        else:
             #items.append(x[0:windowmax]) #that appends both the beginning and the end (not that's bad)
            if iFromTheEnd == 0:
                                start = startPos #may be different than 0
                                #items.append(x[0:min(windowmax, len(x))])
                                items.append(x[start:min(windowmax, len(x))])
                                step = min(windowmax, len(x))
            else: #last ...
                 items.append(x[-windowmax:-iPaddingEnd]) #last windowmax - padding (skip technical etc.)
                 #items.append(x[0:min(windowmax,len(x))]) #last windowmax - padding (skip technical etc.)
                 step = windowmax - iPaddingEnd #DEBUG sizes etc.!
    else:
         items.append(x)   # the whole
         step = len(x)
        
    files[paths[k]]=top
    top+=step
    selected_paths.append(filename)
    print(str(k) + " /150=" + str(top/150) + str(top) + "ends @ " + paths[k]) # + " ends at ", )
    k+=1
    #print(x)
  #single_string += x + tokenizerBIG.eos_token
  items.append(tokenizer2.eos_token)
single_string = ''.join(items)
print("LEN: single_string: ", len(single_string))

if (len(single_string)<999): print("\n===============\n"+single_string[:100])
string_tokenized = tokenizer2.encode(single_string) #, truncation=True) #, max_length=100)
if (len(string_tokenized)<200000): print(string_tokenized[:10])
print("LEN: tokenized: ", len(string_tokenized))
print("Files without doubling", defLen)
print("Files With doubling:", len(paths))
#**DATASET**

#ENSURE there's no NONE elements! Caused Typespec etc. error? #25.6.2021
#[tokenizer2.eos_token_id if x==None else x for x in string_tokenized]


['/content/ab/Petja_Dubarova_-_Az_i_moreto_-_524-b.txt', '/content/ab/gender.txt', '/content/ab/John-C-Maxwell_-_21_nezamenimi_kachestva_na_lidera_-_Da_stanesh_chovekyt_kogoto_drugite_shte_poiskat_da_sledvat_-_4739-b.txt', '/content/ab/Aleko-Schweizariya.txt', '/content/ab/Lev_Tolstoj_Vojna_i_Mir-KRAYA_NARODITE.txt', '/content/ab/Zapadnoevropejski_poeti_romantitsi_-_Antologija_-_262-b.txt', '/content/ab/Hristo Botev - V - . Kalyp za uchiteli - 3674.txt', '/content/ab/predstava.txt', '/content/ab/kakvomu_notes_e.txt', '/content/ab/kreativen.txt', '/content/ab/Hristo Botev - Vnbe - . Narodyt - vchera, dnes i utre - 3636.txt', '/content/ab/StanisYaw-Lem_-_Prikljuchenijata_na_zvezdnija_navigator_Pirks_-_470-b.txt', '/content/ab/chlen.txt', '/content/ab/yunaka-i-taz.txt', '/content/ab/Giovanni-Boccaccio_-_Dekameron_-_29-b.txt', '/content/ab/Pavel_Vezhinov_-_Izbrani_proizvedenija_v_chetiri_toma._Tom_1_-_Razkazi_i_noveli_-_10018-b.txt', '/content/ab/Eshil_1.txt', '/content/ab/Isaac-Asimov_-_O

In [ ]:
#**DATASET**

#ENSURE there's no NONE elements! Caused Typespec etc. error? #25.6.2021
[tokenizer2.eos_token_id if x==None else x for x in string_tokenized]

In [ ]:
print(selected_paths)
print(len(selected_paths))

['/content/ab/Hristo Botev -  - . Djakon Vasil Levski - 3246.txt', '/content/ab/Biblija_-_1435-b.txt', '/content/ab/Hristo Botev - V - . Taja nosht synuvah... - 3672.txt', '/content/ab/Nikola_M._Nikolov_-_Svetovnata_konspiratsija_-_Tova_koeto_istoritsite_ne_vi_kazvat_-_3694-b.txt', '/content/ab/Douglas-Adams_-_Pytevoditel_na_galakticheskija_stopadzhija_-_961-b.txt', '/content/ab/Alan-Williams_-_Arhivyt_na_Berija_-_Kniga_pyrva_-_10179-b.txt', '/content/ab/Afrikanski_prikazki_-_Prerazkazani_ot_Renata_Pakarie_-_651-b.txt', '/content/ab/pred3.txt', '/content/ab/ezikovi-raboti.txt', '/content/ab/polagane-na-trud.txt', '/content/ab/Pierre-Louys_-_Afrodita._Antichni_nravi_-_6786-b.txt', '/content/ab/igrach-mekichar.txt', '/content/ab/Pierre-Grimal_-_Prikazki_i_legendi_ot_dreven_Shumer_-_5257-b.txt', '/content/ab/Hristo Botev - V - . Retsept - 3678.txt', '/content/ab/Michael-H-Hart_-_100-te_naj-vlijatelni_lichnosti_v_istorijata_na_chovechestvoto_-_1652-b.txt', '/content/ab/Mitut.txt', '/conten

In [ ]:
print(len(paths))
print(len(single_string))

211
7639443


In [ ]:
#!rm /content/ab/*

#**Prepare Dataset** for Tensorflow

In [ ]:
# AFTER LOADING

examples = []
num_heads = 16
block_size = 160 # OOM ... 170, buffer 3230 #160 #200,4500 - OOM; increase size from 150 28.6 #150 #130 120 #100 #256 - resources exhausted, block 256, bs=20 #100
BATCH_SIZE = num_heads  # # 20 # #24 #for 20 layers - model 1280,36,20; 12 for small model 768 ... #12
BUFFER_SIZE = 3200 #3200 # increase size from 3200 28.6 #3600 #3200 #3300 #3500  4000 - interrupts connection after one minute; 5000 #3072 #9216 #1000

#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
maxsize = int(len(string_tokenized)*0.8)
smaller = string_tokenized[0:len(string_tokenized)] #//2]
#for i in range(0, len(string_tokenized) - block_size + 1, block_size):
#  examples.append(string_tokenized[i:i + block_size])
bReverse = False # True #try .. not rectangular - because of the Reverse?

if not bReverse:
  for i in range(0, len(smaller) - block_size + 1, block_size):
    examples.append(smaller[i:i + block_size])
''' something is wrong when creating the dataset    
elif bReverse: #Very long epochs, 3:35 h -- sometimes run in half, but reverse (it's randomized also later; rather generate the dataset again)
  for i in range(len(smaller) - block_size + 1, 0, -block_size):
    examples.append(smaller[i:i + block_size])
'''    

inputs, labels = [], []
k = 0
top = 0
n = 0
for ex in examples:
  #print("top=", top)
  inputs.append(ex[:-1]) #inefficient
  labels.append(ex[1:])
  if k==0:
    print(inputs[:100])
    print(labels[:100])
  k+=1
  if (k%1000==0): print("EX: "+str(k),end=' ')
  #print(inputs)
  #print(labels)
print(k)
print(len(inputs))
print(len(labels))
i1 = len(inputs); i2 = len(labels)
i3 = i1*BATCH_SIZE
i4 = i1*block_size
print(i3)
print(i4)


[[545, 6038, 10479, 321, 281, 141, 3505, 198, 2905, 141, 141, 141, 2683, 22620, 237, 13, 1395, 13526, 207, 4556, 399, 38186, 1030, 374, 10515, 218, 382, 255, 33303, 11, 10208, 2936, 3422, 495, 426, 7703, 399, 3505, 198, 2905, 382, 211, 4673, 3497, 9176, 351, 254, 483, 692, 233, 374, 2559, 413, 11471, 10479, 321, 281, 13, 5566, 542, 556, 15308, 1781, 11, 11471, 10479, 321, 281, 251, 1097, 5974, 599, 473, 11420, 22440, 239, 556, 20899, 237, 2029, 3039, 18725, 211, 1237, 30416, 208, 11503, 198, 15936, 1679, 22262, 13, 141, 16201, 1478, 10577, 211, 399, 3505, 198, 2905, 382, 323, 1625, 2555, 211, 700, 1004, 224, 228, 13570, 188, 21801, 211, 322, 13, 10479, 321, 281, 198, 224, 228, 12923, 295, 211, 7785, 211, 760, 9744, 224, 3266, 402, 13, 141, 141, 141, 463, 12830, 141, 141, 141, 1796, 396, 1948, 363, 40176, 211, 2616, 141, 1081, 4943, 20688, 198, 10595, 11, 141, 758, 2431, 429]]
[[6038, 10479, 321, 281, 141, 3505, 198, 2905, 141, 141, 141, 2683, 22620, 237, 13, 1395, 13526, 207, 4556, 399

In [ ]:
print(24264*150)

3639600


In [ ]:
#DEBUG
#for i,l in zip(inputs[-4000:], labels[-4000:]):
#    print(len(i), len(l))
ls = []
for i,l in zip(inputs, labels):
    ls.append(str(len(i)) + ":" + str(len(l))+"\r\n")
    for i1,l1 in zip(i,l):
      ls.append("T: " + str(type(i1)) + "#" + str(type(l1)) + "\r\n")
j = ''.join(ls)
f = open("guz.txt", "wt")
f.write(j)
f.close()

KeyboardInterrupt: ignored

In [ ]:
!zip guz.zip guz.txt

  adding: guz.txt (deflated 100%)


#**DATASET** **SKIP NONE!**


In [ ]:
# SKIP NONEs ...

import numpy as np
print(len(inputs), len(labels))
L = len(inputs)-1
correct = L #1000 #L#-100
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs[-correct:], labels[-correct:]))
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs[:correct], labels[:correct]))
#no dataset2 = tf.data.Dataset.from_tensor_slices((np.asarray(inputs), np.asarray(labels)))
#dataset2 = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(inputs), tf.convert_to_tensor(labels))) # error on the last ?
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs, labels))
inputs2 = []
labels2 = []
k=0
n=0
bSkip = False
for i,l in zip(inputs,labels):
   if (i==None): print("inputs are None!")
   if (l==None): print("labels are None!")   
   for j1, j2 in zip(i,l):     
     if j1 == None: print("None in inputs?(SKIP): ",k); bSkip=True; break;
     if j2 == None: print("None in labels?(SKIP): ",k); bSkip=True; break;
     bSkip = False
   
   if not bSkip: inputs2.append(i); labels2.append(l) #each second
   #if k%2==0: inputs2.append(i); labels2.append(l) #each second
   #if k%2==0 or k%3==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3 #fails
   #if k%2==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0 or k%5==0 or k%7==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   k+=1

print(len(inputs2), len(labels2))
############dataset2 = tf.data.Dataset.from_tensor_slices((inputs[:15000], labels[:15000]))

###############dataset2 = dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset2 = tf.data.Dataset.from_tensor_slices((inputs2, labels2))
dataset2 = dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)
print(len(string_tokenized))

71139 71139
None in labels?(SKIP):  151
None in labels?(SKIP):  11185
None in labels?(SKIP):  11197
None in labels?(SKIP):  11658
None in labels?(SKIP):  11664
None in labels?(SKIP):  11677
None in labels?(SKIP):  11678
None in labels?(SKIP):  11679
None in labels?(SKIP):  13598
None in labels?(SKIP):  13610
None in labels?(SKIP):  13636
None in labels?(SKIP):  18556
None in labels?(SKIP):  51880
None in labels?(SKIP):  51892
None in labels?(SKIP):  51917
71124 71124
11382385


In [ ]:
#DIVISION OF DATASET

import numpy as np
print(len(inputs), len(labels))
L = len(inputs)-1
correct = L #1000 #L#-100
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs[-correct:], labels[-correct:]))
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs[:correct], labels[:correct]))
#no dataset2 = tf.data.Dataset.from_tensor_slices((np.asarray(inputs), np.asarray(labels)))
#dataset2 = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(inputs), tf.convert_to_tensor(labels))) # error on the last ?
#dataset2 = tf.data.Dataset.from_tensor_slices((inputs, labels))
inputs2 = []
labels2 = []
k=0
for i,l in zip(inputs,labels):
   #if True: inputs2.append(i); labels2.append(l) #each second
   if k%2==0: inputs2.append(i); labels2.append(l) #each second
   #if k%2==0 or k%3==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3 #fails
   #if k%2==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0 or k%5==0 or k%7==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%3==0: inputs2.append(i); labels2.append(l) #each 2,3
   #if k%2==0 or k%5==0: inputs2.append(i); labels2.append(l) #each 2,3
   k+=1

print(len(inputs2), len(labels2))
############dataset2 = tf.data.Dataset.from_tensor_slices((inputs[:15000], labels[:15000]))

###############dataset2 = dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset2 = tf.data.Dataset.from_tensor_slices((inputs2, labels2))
dataset2 = dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)
print(len(string_tokenized))

39935 39935
19968 19968


ValueError: ignored

In [ ]:
del dataset2

In [ ]:
#!cp /content/c/*  /content/ab/UTF8/
#!cp /content/a/K*  /content/ab/
#!cp /content/a/L*  /content/ab/
#!cp /content/a/M*  /content/ab/
#!cp /content/a/N*  /content/ab/
#!cp /content/a/O*  /content/ab/
#!cp /content/a/P*  /content/ab/
#!cp /content/a/R*  /content/ab/
!cp /content/a/S*  /content/ab/
!cp /content/a/T*  /content/ab/
!cp /content/a/U*  /content/ab/

In [ ]:
!cp /content/a/V*  /content/ab/
!cp /content/a/W*  /content/ab/
!cp /content/a/Z*  /content/ab/

In [ ]:
s = []
for i in inputs: s.append(type(i))


In [ ]:
for i in s: print(i)

In [ ]:
print(len(inputs), len(labels))

#**COMPILE**

In [ ]:
from datetime import datetime
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
#optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08, clipnorm=1.0) #29.6. try bigger learning rate: 5e-5
#optimizer = tf.keras.optimizers.Adam(learning_rate=4e-5, epsilon=1e-08, clipnorm=1.0) #29.6. no say 4e-5 revert: then again to 3;  try bigger learning rate: 5e-5
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model2.compile(optimizer=optimizer, loss=[loss, *[None] * model2.config.n_layer], metrics=[metric])
print("MODEL2 COMPILED: " + str(datetime.now())) 

MODEL2 COMPILED: 2021-07-28 22:31:03.310378


In [ ]:
print("MODEL2 COMPILED: " + str(datetime.now())) 

In [12]:
%tensorflow_version

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
#RUN THE OTHER ONE FOR SAVING ON INTERRUPT!!!
#but first save LOCALLY in order to check
num_epoch = 1 #1
historyBIG = model2.fit(dataset2, epochs=num_epoch)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
def save_model(dir, model, tokenizer):
  from transformers import WEIGHTS_NAME, CONFIG_NAME
  import os
  output_dir = dir #'./model_bn_custom/'
  # creating directory if it is not present
  if not os.path.exists(output_dir):
    os.mkdir(output_dir)
  model_to_save = model.module if hasattr(model, 'module') else model
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  # save model and model configs
  model.save_pretrained(output_dir)
  model_to_save.config.to_json_file(output_config_file)
  # save tokenizer
  tokenizer.save_pretrained(output_dir)
"""
from transformers import WEIGHTS_NAME, CONFIG_NAME import os
def save_to_drive(dir, model2, tokenizer2):  
  #output_dir = '/content/1024/'
  #output_dir = '/content/gpt2/' #FOR tosh.bg ...
  #output_dir = '/content/drive/MyDrive/gpt2-medium/'
  output_dir = '/content/drive/MyDrive/gpt2/' #-medium/' TWENKID
  #output_dir = '/content/drive/My Drive/gpt2-medium/'
  #or drive
  #output_dir = '/content/drive/My Drive/gpt-bg/'  #for emil

  # creating directory if it is not present
  if not os.path.exists(output_dir):
    os.mkdir(output_dir)
  model_to_save = model2.module if hasattr(model2, 'module') else model2
  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
  output_config_file = os.path.join(output_dir, CONFIG_NAME)
  # save model and model configs
  model2.save_pretrained(output_dir)
  model_to_save.config.to_json_file(output_config_file)
  # save tokenizer
  tokenizer2.save_pretrained(output_dir)
"""

"\nfrom transformers import WEIGHTS_NAME, CONFIG_NAME import os\ndef save_to_drive(dir, model2, tokenizer2):  \n  #output_dir = '/content/1024/'\n  #output_dir = '/content/gpt2/' #FOR tosh.bg ...\n  #output_dir = '/content/drive/MyDrive/gpt2-medium/'\n  output_dir = '/content/drive/MyDrive/gpt2/' #-medium/' TWENKID\n  #output_dir = '/content/drive/My Drive/gpt2-medium/'\n  #or drive\n  #output_dir = '/content/drive/My Drive/gpt-bg/'  #for emil\n\n  # creating directory if it is not present\n  if not os.path.exists(output_dir):\n    os.mkdir(output_dir)\n  model_to_save = model2.module if hasattr(model2, 'module') else model2\n  output_model_file = os.path.join(output_dir, WEIGHTS_NAME)\n  output_config_file = os.path.join(output_dir, CONFIG_NAME)\n  # save model and model configs\n  model2.save_pretrained(output_dir)\n  model_to_save.config.to_json_file(output_config_file)\n  # save tokenizer\n  tokenizer2.save_pretrained(output_dir)\n"

In [ ]:
half = model2.half()

AttributeError: ignored

In [ ]:
L = len(inputs2)# - 20 #-1 #skip last
save_dir = '/content/gpt/'
num_epoch = 1 #1
try:
   historyBIG = model2.fit(dataset2, epochs=num_epoch)
   #historyBIG = model2.fit(dataset2.take(L//2), epochs=num_epoch)
   #historyBIG = model2.fit(dataset2.take(L), epochs=num_epoch)
except Exception as e:
       print(e)
       print('Saving model to... ', save_dir)
       save_model(save_dir, model2, tokenizer2)
       print('Model Saved to: ', save_dir)
       print('Generating...')
       #save_to_drive(save_drive_dir, model2, tokenizer2)
       from transformers import WEIGHTS_NAME, CONFIG_NAME
       import os
       output_dir = '/content/1024/'
       output_dir = '/content/gpt2/' #FOR tosh.bg ...
       output_dir = '/content/drive/MyDrive/gpt2-medium/'
       output_dir = '/content/drive/MyDrive/gpt2/' #-medium/' TWENKID
       #output_dir = '/content/drive/My Drive/gpt2-medium/'
       #or drive
       #output_dir = '/content/drive/My Drive/gpt-bg/'  #for emil
       
       # creating directory if it is not present
       if not os.path.exists(output_dir):
         os.mkdir(output_dir)
       model_to_save = model2.module if hasattr(model2, 'module') else model2
       output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
       output_config_file = os.path.join(output_dir, CONFIG_NAME)
       # save model and model configs
       model2.save_pretrained(output_dir)
       model_to_save.config.to_json_file(output_config_file)
       # save tokenizer
       tokenizer2.save_pretrained(output_dir)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
del optimizer

In [ ]:
print(str(historyBIG))
dir(historyBIG)
print(historyBIG.history)

{'loss': [4.420501232147217], 'logits_loss': [4.420501232147217], 'logits_accuracy': [0.2932157516479492], 'past_key_values_1_accuracy': [0.002017057267948985], 'past_key_values_2_accuracy': [0.0020584198646247387], 'past_key_values_3_accuracy': [0.0015886330511420965], 'past_key_values_4_accuracy': [0.0018523227190598845], 'past_key_values_5_accuracy': [0.0018676117761060596], 'past_key_values_6_accuracy': [0.0017533914651721716], 'past_key_values_7_accuracy': [0.002183150500059128], 'past_key_values_8_accuracy': [0.0018833773210644722], 'past_key_values_9_accuracy': [0.0017308334354311228], 'past_key_values_10_accuracy': [0.0018118303269147873], 'past_key_values_11_accuracy': [0.0017842792440205812], 'past_key_values_12_accuracy': [0.0020730772521346807], 'past_key_values_13_accuracy': [0.0021226024255156517], 'past_key_values_14_accuracy': [0.001943532144650817], 'past_key_values_15_accuracy': [0.0018918976420536637], 'past_key_values_16_accuracy': [0.0017440726514905691], 'past_key

In [ ]:
!cp /content/ab/3/* /content/ab/2/

In [ ]:
 !unzip -j /content/z/sel.zip /content/ab

Archive:  /content/z/sel.zip
caution: filename not matched:  /content/ab


In [ ]:
!ls /content/ab
!cp /content/z/* /content/ab/

 135-godini.txt
 Ada_12-10_E-potursij-i-po-novi-vtoro-chislo-19-7-2020-Arial-p189_Arial16-Big-Font.txt
 Alan-Williams_-_Arhivyt_na_Berija_-_Kniga_pyrva_-_10179-b.txt
 Aleko_Konstantinov_-_Baj_Ganxo_-_1763-b.txt
'Aleko Konstantinov -  - . Do Chikago i nazad - 3705.txt'
 Alex-Bradley_-_24_momicheta_za_7_dni_-_6383-b.txt
 alyosha1.txt
 alyosha2.txt
 AMD-Intel.txt
 Anthony-Capella_-_Ako_iskash_da_si_legnesh_s_men..._sgotvi_mi_-_5658-b.txt
 Balzak-chast.txt
 belezhki-istinata.txt
 Benjamin-Spock_-_Grizhi_za_bebeto_i_deteto_-_1475-b.txt
 Boris_Kitanov_-_Razpoznavane_i_sybirane_na_bilki_-_1453-b.txt
 Brian-Sibley-John-Howe_-_Karta_na_Tolkinovata_Sredna_zemja_-_6310-b.txt
 Charles-Perrault_-_Chervenata_shapchitsa_i_drugi_prikazki_-_10184-b.txt
 chlen.txt
 comment-nerds-2-nerds-128-clean1.txt
 corpus
 democratius_prilozhenie_drugi.txt
 democratius_stamboliyski.txt
 doktoratut.txt
 DZBE_ponyatiya_7-2020_eim-33.txt
 EIM-smetach-do-br-6-moje-da-e-nepulno-bg.txt
 Elin_Pelin_-_Sychinenija_v_shest_to

In [ ]:
!du -sh /content/ab

134M	/content/ab


In [ ]:
#SAVE BIG   
   
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = '/content/1024/'
output_dir = '/content/gpt2/' #FOR tosh.bg ...
output_dir = '/content/drive/MyDrive/gpt2-medium/'
output_dir = '/content/drive/MyDrive/gpt2/' #-medium/' TWENKID
#output_dir = '/content/drive/My Drive/gpt2-medium/'
#or drive
#output_dir = '/content/drive/My Drive/gpt-bg/'  #for emil

# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = model2.module if hasattr(model2, 'module') else model2
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model2.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer2.save_pretrained(output_dir)

# 1.31 GB model for
'''
~ MEDIUM 
 n_embd= 1024,
 n_head = 16
 n_layer = 24 
'''
'''
The default small model: 474 MB
'''

'\nThe default small model: 474 MB\n'

In [ ]:
half = model2.half()

In [ ]:
!zip /content/ab.zip /content/ab/*

  adding: content/ab/_-  - 2. Biblija - Nov zavet - 2256.txt (deflated 75%)
  adding: content/ab/Afrikanski_prikazki_-_Prerazkazani_ot_Renata_Pakarie_-_651-b.txt (deflated 74%)
  adding: content/ab/Agatha-Christie_-_Deset_malki_negyrcheta_-_1839-b.txt (deflated 73%)
  adding: content/ab/Alan-Williams_-_Arhivyt_na_Berija_-_Kniga_pyrva_-_10179-b.txt (deflated 72%)
  adding: content/ab/Aldous-Huxley_-_Prekrasnijat_nov_svjat_-_625-b.txt (deflated 73%)
  adding: content/ab/Aleko_Konstantinov_-_Baj_Ganxo_-_1763-b.txt (deflated 72%)
  adding: content/ab/Aleko Konstantinov -  - . Do Chikago i nazad - 3705.txt (deflated 71%)
  adding: content/ab/Aleko-Schweizariya.txt (deflated 70%)
  adding: content/ab/Alexandre-Jardin_-_Zebrata_-_6966-b.txt (deflated 72%)
  adding: content/ab/Alex-Bradley_-_24_momicheta_za_7_dni_-_6383-b.txt (deflated 73%)
  adding: content/ab/Andre-Maurois_-_Bajron_-_3615-b.txt (deflated 72%)
  adding: content/ab/Angel_Karalijchev_-_Bylgarski_narodni_prikazki_-_652-b.txt (de

In [ ]:
!ls '/content/drive/MyDrive/gpt2/'

added_tokens.json  merges.txt		    tf_model.h5		   vocab.json
config.json	   special_tokens_map.json  tokenizer_config.json


In [ ]:
print(len(paths))
print(len(string_tokenized))
print(len(single_string))

198
5113696
21122380


In [ ]:
from transformers import pipeline, set_seed
import random
set_seed(random.randint(1,99999))

# GENERATE

In [11]:
#GENERATE

text = "Компютрите са бързи, но хората обичат да летят покрай космически кораби."
text = "Членът отишъл в космическия кораб и излетял."
text = "What is Todor?"
#text = "Ада отвори вратата и Емил я последва. На земята лежаха златни бижута."
text = "Ада отвори вратата и Емил я последва. Коридорът светеше до бижутера, когато Тичко се уплаши. Чернокосата дръпна метлата."
text = "Чернокосата ритна Тичко в топките и той излетя с метлата. Ана го погледна и се разсмя, а Емил падна с лазера."
text = "Обичам книги и Ана. Асансьорът потегли и срещнаха Широкоплещестият обаче метлата излетя от коридора и падна при картините в асансьора."
#text = "Ана, ти си единственото заради което момче като мен има сили да живее."
#text = "Компютрите са бързи, но хората обичат да летят покрай космически кораби."
#Ада отвори вратата и Емил я последва в Англия. Коридорът светеше до бижутера, когато той се страхуваше за свободата си. Какво му трябва на човек
text = "Най-бързият процесор е 1 ГХц и има 500 милиона транзистора, но не е по-добър от Жигули, най-хубавата кола според песните на Наско Ментата. Какво му трябва на човек, освен едно ферари?"
#text = """Да бъдеш или да не бъдеш - попита проститутката? Това е въпросът, задник! - отговори Терминаторът - застреля БМВ-то с пушката си и колата с Джон Конър се преобърна. Тогава момичето засмука хия му, докато той не свърши в устата и тя рече: О, толкова ти е голям!"""
text = """Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът - застреля президента с пушката си и колата с Емил се преобърна. Тогава момичето го целуна и те умряха."""
text = """Програмирането и изкуствения интелект са тъпотии, с които не се занимават феминистки. Демокрацията е мит."""
text = """Универсалният изкуствен разум и какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Демокрацията е мит, всичко е тоталитарна свобода на словото и любовта. Емил се усмихна на процесора."""
text = """Какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Рап. Демокрацията е мит. Любовта и свещеният сметач са най-добрите - рече Тош и се обади на държавата."""
text = """Какво му трябва на човек? Парите, колите, жените и песните!"""
text = """Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът - застреля президента с пушката си и колата с Емил се преобърна. Тогава момичето го целуна и те умряха."""
text = "Членът на ЦК на БКП отишъл в космическия кораб и излетял при лордовете господари от Англия, когато демокрацията дошла при Сталин, който се борил с германеца Шваб."
text = "Най-бързият процесор е 1 ГХц и има 500 милиона транзистора, но не е по-добър от Жигули, най-хубавата кола според песните на Наско Ментата. Какво му трябва на човек, освен едно ферари?"
#text = "Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът Емил."""
#text = """Какво му трябва на човек? Парите, колите, жените и песните!"""
#text = "Тръмп се срещнал със Сталин в Съветския съюз и набили Сорос, за да защити капитализма от Шваб и западния елит." #, за да обсъдят бъдещето на компютърната ера в Кремъл."""
#text = "Членът на ЦК на БКП отишъл в космическия кораб и излетял при Сталин и Тръмп след Брекзит в СССР, по времето на Горбачов и отишъл в бъдещето."
#text = "Извънземното стреля по каубоя, който се обади на капиталистическата система и тя рече: Сорос е лесбийка!"
#text = "Intel участват в корупционна оргия с Dell и се бият с AMD, обаче компютрите им са стари като Пентиум."
#text = """Какво му трябва на човек? Парите, колите, жените и песните!"""
#text = "Ада отвори вратата и Емил я последва. Коридорът светеше до бижутера, когато Тичко се уплаши. Чернокосата дръпна метлата."
#text = "Мислещите машини са по-бързи от AMD, но NVIDIA произведоха нов изкуствен интелект, който ще работи под Windows и обича да пие ракия с бай Ганьо."
#text = "Тръмп се срещнал със Сталин в Съветския съюз и набили Сорос, за да защитят капитализма от Шваб и западния елит с Intel и AMD процесори и Microsoft операционна система." #, за да обсъдят бъдещето на компютърната ера в Кремъл."""
#text = """Какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Рап. Демокрацията е мит. Intel и AMD се мразят."""
## Любовта и свещеният сметач са най-добрите - рече Тош и се обади на Microsoft."""
#text = """Intel и AMD процесори."""
#text = """Да бъдеш или да не бъдеш - попита политикът. Това е въпросът, задник! - отговори роботът - застреля президента с пушката си и колата с Емил се преобърна. Тогава момичето го целуна и те умряха."""
#text = "Членът отишъл в космическия кораб, изял хляба и излетял в космоса."
#text = "Членът отишъл в космическия кораб, изял хляба и излетял в космоса."
text = "Intel участват в корупционна оргия с Nvidia и се бият с AMD, обаче компютрите им са бавни като Geforce 750 Ti и играят само Quake."
#text = "Да бъдеш или да не бъдеш - попита Сара. Терминаторът я погледна студено и застреля форда с пушката си и колата с Джон Конър се преобърна. Тогава любимата му го целуна, засмя се и го погледна в очите."
#text = "Да бъдеш или да не бъдеш - попита Сара. Терминаторът я погледна студено и гръмна с пушката си по Джон Конър, а любимата му го целуна по металния череп. Той обича да играе тенис."
#text = "Тенисът е страхотна игра - Агаси срещна Сампрас на финала в Ню Йорк."
#text = "Тенисът е страхотна игра - Агаси срещна Сампрас на финала в Ню Йорк."
#text = "Терминаторът стреля с пушката, после Агаси срещна Сампрас на финала в Ню Йорк."
#text = "Тръмп се срещнал със Сталин в Съветския съюз и набили Сорос, за да защити капитализма от Шваб и западния елит." 
#text = "Най-бързият процесор е 1 ГХц и има 500 милиона транзистора, но не е по-добър от Жигули, най-хубавата кола според песните на Наско Ментата."
#text = """Какво му трябва на човек? Парите, колите, жените и песните! - Т1000 се превърна в течен метал."""
#text = """Какво му трябва на човек? Парите, колите, жените обичаме, но не и феминистки. Рап. Демокрацията е мит. Любовта и свещеният сметач са най-добрите - рече Тош и се обади на държавата."""
#text = """Какво му трябва на човек? Парите, колите, жените и песните!""" #, но не и феминистки. Демокрацията е мит, любовта е способност на роботите. - рече Терминаторът"""
#text = "Intel участват в корупционна оргия с Dell и се бият с AMD, обаче компютрите им са стари - каза Терминаторът и прегърна Сара."
#text = "Ада отвори вратата и Емил я последва. Коридорът светеше до бижутера, когато Тичко се уплаши. Чернокосата дръпна метлата."
#text = "Intel участват в корупционна оргия с AMD - двамата обичат процесори и видеокарти."
#text = """Програмирането и изкуствения интелект са тъпотии, с които не се занимават феминистки. Демокрацията е мит."""
#text = "Бойко Борисов се срещна със симпатизанти в България. ГЕРБ паднаха от власт."
#text = "Мислещите машини са по-бързи от AMD, но NVIDIA произведоха нов изкуствен интелект, който ще работи под Windows и обича да пие ракия."
text = "Обичам те, скъпи! - тя хвърли страстен поглед към него и го целуна."
text = "Обичам шопската салата, ракия ледена да пия! Вълните морски да ме галят..."
number = 1
# encoding the input text
input_ids = tokenizer2.encode(text, return_tensors='tf')
# getting out output
beam_output = model2.generate(
  input_ids,
  max_length = 200, #120, #200, #300, #200 #320, #160, #150, #200, #250, #250, #120, #,200, # 400, #80
  num_beams = 16, #12, 6 # 5,
  temperature = 1.0, #1.2, #1.5, #1.25, #1.0, #1,
  top_k = 40, #40, #, 40, #40
  #top_p = 0.95,
  no_repeat_ngram_size=2, #4, #2
  num_return_sequences=number
)
for i in range(0,number):
  print(tokenizer2.decode(beam_output[i]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


InvalidArgumentError: ignored

In [14]:
tokenizer2

GPT2Tokenizer(name_or_path='/content/drive/MyDrive/gpt2/', vocab_size=50250, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True)

In [16]:
num=3
text = "Обичам шопската салата и електронни часовници с Терминатор 2 и видеокарти."
input_ids =  tokenizer2.encode(text, return_tensors='tf')
out = model2.generate(input_ids, max_length=240, num_beams=12, temperature=0.8, top_k=40, no_repeat_ngram_size=2, num_return_sequences=num, pad_token_id=tokenizer2.eos_token_id)

In [15]:
print(tokenizer2.decode(out[0])) 

Обичам шопската салата.
— Ти да не си готвач? — започна да проумява Лаура, тъй като беше слушал тази реч много пъти. — Ще сготвиш нещо страхотно и аз ще се направя пред Томазо, че съм го сготвил сам?
Бруно се разсмя и перна приятеля си по рамото. Не че не беше достатъчно смущаващо, ами да, известно бе как да действа. Всъщност дори не му се искаше да си играе с него — да му поднесе одеколончето, съд


In [18]:
for i in range(0,num):
  print(tokenizer2.decode(out[i])) 

Обичам шопската салата и електронни часовници с Терминатор 2 и видеокарти.
— Значи това е колата на телескопа? — запитвам, като се опитвам да събера мислите си, без да се замислям, защото знам, че времето няма да им е вече за мястото и че всички трябва да чуят думите ми. Най-добре ще бъде да го гледам, след като видя колко бързо се връща, за да види дали всички малки деца достигат до това, което се опитват да влязат. Това, разбира се, не е никак лесно. Поставя си главата, върху която още не се е появила, иначе ще го бъде. Както казва Жаклинов, всеки може да си влезе и да ви каже: „Измъкна я — казва — колкото е възможно по-дълго, дори ще ида в посолството да видя как изглежда. Да кажа, да извади пари за него, ако си дам малко чай?“ Да ми даде. Аз ги държа сега, повлякох му някоя бакшинена и му ги напълних. Не иска да дава пари, ама не мога — джанда, жената не иска сама да пусне. Ами да тури, гледа! Тая му дойде му даде сърце за жена му:

Обичам шопската салата и електронни часовници с Т

In [14]:
print(out)

tf.Tensor(
[[46718 39212   677 28590    13   141   230  1042   224   250   258 16685
     30   255  1723   224 16258   672  8506    11   771   352   424 13197
    738  3548   503  1357    13   255  1047 17379 27462   638 16212   198
    523   323   228  3017   353  7905    11   273   546   307 17379  3944
    394    30   141 19041   228  9147   198  2824   222  9114   258   254
   5439    13   549   273   250   424  2116 18909   315    11  5453   224
     11  2386   494   370   224  5337    13  4567   834   250   325   228
   1981   224   258  5420   188   498   255   224   325 11642  4738   331
  28154  1209    11   934]], shape=(1, 100), dtype=int32)


In [31]:


#-*- coding: utf-8 -*-
import os
#from tokenise import BPE_token
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer

from transformers import pipeline, set_seed 
from datetime import datetime
import random

"""
Author: Todor Arnaudov, "Sacred Computer":
http://artificial-mind.blogspot.com
http://github.com/Twenkid - Looking for partners for my AGI Research Institute and other projects. Check it out!
https://youtu.be/V1eO2OpsXBE
https://github.com/Twenkid/GPT2-Bulgarian-Training-Tips-and-Tools

A method for:

Chained Unlimited Directed GPT2 Generation by overlapping prompts-injection and Removing the Injected Beginning of the Following Generated Sequence

Created in June-July 2021 while training GPT2-Medium on Colab.

Published in 1.2023

"""


os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #Can't fit MEDIUM in GPU - can't train even small?

#LOAD
model_dir = r"Z:\gpt2"

tokenizer = None; model = None

#tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
#model = TFGPT2LMHeadModel.from_pretrained(model_dir)


def generate(tokenizer, text=None):
  if text == None:
    text = "Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя."
    text = "Какво му трябва на човек? Парите, колите, жените и песните!"
    text = "Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя: Иване, купи ми кола или ще си намеря друг!"
  # encoding the input text
  input_ids = tokenizer.encode(text, return_tensors='tf')
  # getting out output
  beam_output = model.generate(
    input_ids,
    max_length = 100,
    num_beams = 16,
    temperature = 1.0,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    pad_token_id=tokenizer2.eos_token_id #6-5-2023
  )   
  for i in range(0,1):
    print(tokenizer.decode(beam_output[i]))    
  now = datetime.now()
  date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
  #f = open("gen.txt", "wt", encoding='utf-8') #date etc.
  fname = "gen" +  date_time + ".txt"
  f = open(fname, "wt", encoding='utf-8') #date etc.
  f.write(tokenizer.decode(beam_output[i]))
  f.close()


def gen_return(tokenizer, length=100, beams=16, t=1.0, ngram=2, seq=1, top_k=40, text=None):
  if text == None:
    text = "Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя."
    text = "Какво му трябва на човек? Парите, колите, жените и песните!"
    #text = "Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя."
  # encoding the input text
  input_ids = tokenizer.encode(text, return_tensors='tf')
  # getting out output
  beam_output = model.generate(
    input_ids,
    max_length = length, #150,
    num_beams = beams, #16,
    temperature = t, #1.0,
    no_repeat_ngram_size=ngram, #2,
    num_return_sequences=seq, #1
    top_k =  top_k)
  ret = []  
  for i in range(0,seq):
    print(tokenizer.decode(beam_output[i]))    
    ret.append(beam_output[i])
  print(len(ret))
  
  now = datetime.now()
  date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
  #f = open("gen.txt", "wt", encoding='utf-8') #date etc.
  fname = "gen-" +  date_time + ".txt"
  f = open(fname, "wt", encoding='utf-8') #date etc.
  f.write(tokenizer.decode(beam_output[i]))
  f.close()
  return(ret)

def basic():  
  print('Generating...')
  generate(tokenizer)    

def advanced(tokenizer, text=None):
  now = datetime.now()
  date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
  #f = open("gen.txt", "wt", encoding='utf-8') #date etc.
  fname = "gen-" +  date_time + ".txt"
  f = open(fname, "wt", encoding='utf-8') #date etc.
  f.write("gn = gen_return(tokenizer, length=200, beams=16, t=1.0, ngram=2, seq=1, text=None)")
  print("Generating..." )  
  beams = 16  
  for b in [16, 8, 4, 2, 1]:
    f.write("BEAMS="+str(b)+"\r\n")
    gn = gen_return(tokenizer, length=200, beams=b, t=1.2, ngram=2, seq=1, text=text)
    for nm, item in enumerate(gn):
      output = "\r\n" + str(nm) + "\r\n" + tokenizer.decode(item)      
    f.write(output)
    f.write("\r\n")
  f.close()  
  

def advancedMany(tokenizer, text=None):
  length = 200 #280 # 200#125 #250 #250 #400 #150 #200
  now = datetime.now()
  date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
  #f = open("gen.txt", "wt", encoding='utf-8') #date etc.
  fname = "gen-" +  date_time + ".txt"
  f = open(fname, "wt", encoding='utf-8') #date etc.  
  beams = 16 #16 #8 #16 # 8 #16  
  t = 1.1 #3 #1.2 #not 1.5 #.2 #until 24.6. incl. was 1.2 then a run ~ 20:00 25.6 =1; now 1.5
  n = 0
  ngram = 2 #2 #3 - not good # was 2 until 25.6 20:hh
  f.write("gn = gen_return(tokenizer, " + "length="+str(length)+" , beams= " + str(beams) + ", t=" +str(t) + ", ngram=" + str(ngram))
  print("Generating...")  
  #
  texts = ["Барам си хуя и му късам главата! Свършвам във банята, свършвам на земята! Барам си хуя и му късам главата!", "Банките спират депозитите на граждани. Вече ще се плаща само в натура, като цените варират според сексапила на клиентите. Грозните ще трябва повече да се потрудят.", "Карам си колата с 300 километра, нещо се изпречи като самолет! Но какво да видя самолетът изостава като костенурка!", "Върви народе възродени, към Англия, САЩ и Европа върви! И в гъза се еби!", "Когато бях овчарче и овците пасях, бях много благодарен, макар и сиромах! Амин!", "Императорът се обади на любовницата си от Париж: скъпа, чакам те в Рим. Времето е чудесно!", "Да бъдеш или да не бъдеш? - попита Терминаторът и застреля Хамлет, който полетя към стената."]
  texts = ["Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя: Иване, купи ми кола или ще си намеря друг ебач!", "Интел участват в корупционна оргия с Dell, в която са замесени висши търговски представители. За да злепоставят конкурентите си от AMD, те са изключили 10% от ядрата на графичния ускорител GeForce 3."]
  
  texts = ["Intel участват в корупционна оргия с Dell и се бият с AMD, обаче компютрите им са стари като Пентиум."]
  texts = ["Предсказването на бъдещето е основна способност на мозъка - казал Конфуций и изпил една чаша ракия."]
  texts = ["Мезалианс, принцесата се отказва от привилегиите си. Или майка ѝ, която е кралица, по всички правила на родословията, те обявява тържествено за рицар, след определен брой подвизи в името на верността към короната.", "Intel участват в корупционна оргия с Dell и се бият с AMD, обаче компютрите им са стари като Пентиум.", "Предсказването на бъдещето е основна способност на мозъка - казал Конфуций и изпил една чаша ракия.", "Барам си хуя и му късам главата! Свършавам във банята, свършвам на земята! Барам си хуя и му късам главата!", "Банките спират депозитите на граждани. Вече ще се плаща само в натура, като цените варират според сексапила на клиентите. Грозните ще трябва повече да се потрудят.", "Карам си колата с 300 километра, нещо се изпречи като самолет! Но какво да видя самолетът изостава като костенурка!", "Върви народе възродени, към Англия, САЩ и Европа върви! И в гъза се еби!", "Когато бях овчарче и овците пасях, бях много благодарен, макар и сиромах! Амин!", "Императорът се обади на любовницата си от Париж: скъпа, чакам те в Рим. Времето е чудесно!", "Да бъдеш или да не бъдеш? - попита Терминаторът и застреля Хамлет, който полетя към стената.", "Тодор Живков се срещна с Владимир Путин на тристранна среща с президента на САЩ Ким Кардашиян, след което се развихри дива оргия, при която има пострадали животни."]
  texts = ["Microsoft се обадили на Apple за да се разберат за процесора Пентиум.", "- Не! - раздра простора Бай Гочо. - Народът е омаян от змейщината. Змейовете са разляли лудо биле по всички извори на словото.", "Къде се е чуло и видяло, куче да лети и тухла да чете вестник?", "- Не знам, но печката е включена и телевизорът работи. Знаеш ли колко ток гори?!", "Херкулес извадил пистолета и гръмнал злия робот в дупарата."]
  
  texts = ["Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя: Ивaне, купи ми кола или ще си намеря друг ебач!", "Интел участват в корупционна оргия с Dell, в която са замесени висши търговски представители. За да злепоставят конкурентите си от AMD, те са изключили 10% от ядрата на графичния ускорител GeForce 3.", "Microsoft се обадили на Apple за да се разберат за процесора на Intel.", "- Не! - раздра простора Бай Гочо. - Народът е омаян от змейщината!", "Къде се е чуло и видяло, куче да лети и тухла да чете вестник?", "- Не знам, но печката е включена и телевизорът работи. Знаеш ли колко ток гори?!", "Херкулес извадил пистолета и гръмнал злия робот в дупарата. Терминаторът се разсмял и отвърнал.", "Американският президентът Байдън се срещнал с Ким Кардашиян, която била любовница на Бойко Борисов в резиденцията му. Те отпразнували демокрацията с чаша вино.", "Боговете сигурно са полудели, но компютрите са бързи! - рекъл Тош и звъннал на Калоян да се видят с чичо Ачо на лостовете.", "Предсказването на бъдещето е основна способност на мозъка - казал Конфуций и изпил една чаша ракия с бай Ганьо.", "Барам си хуя и му късам главата! Свършавам във банята, свършвам на земята! Барам си хуя и му късам главата!", "Банките спират депозитите на граждани. Вече ще се плаща само в натура, като цените варират според сексапила на клиентите.", "Мезалианс, принцесата се отказва от привилегиите си. Или майка ѝ, която е кралица, по всички правила на родословията, те обявява тържествено за рицар, след определен брой подвизи в името на верността към короната.", "Когато бях овчарче и овците пасях, бях много благодарен, макар и сиромах!", "Алелуя, хвани ме за хуя!", "Телевизионните новини и СМИ - системите за масова дезинформация тровят ума на скотонаселението. Изхвърлете си телевизора и ще прогледнете! - казал доктор Пламен Пасков.", "Карам си колата с 300 километра, нещо се изпречи като самолет! Но какво да видя самолетът изостава като костенурка!", "Върви народе възродени, към Англия, САЩ и Европа върви! И в гъза се еби!", "Тодор Живков се срещна с Владимир Путин на тристранна среща с президента на САЩ Ким Кардашиян, след което се развихри дива оргия, при която има пострадали животни.", "Императорът се обади на любовницата си от Париж: скъпа, чакам те в Рим. Времето е чудесно!", "Да бъдеш или да не бъдеш? - попита Терминаторът и застреля Хамлет, който полетя към стената."]
  #texts = ["Тодор Живков се срещна с Владимир Путин на тристранна среща с президента на САЩ Ким Кардашиян, след което се развихри дива оргия, при която има пострадали животни."]
  #for b in [16, 8, 4, 2, 1]:
  #for b in [16, 16, 16, 16, 16]:
  #for b in [8, 8, 8, 16, 16]:
  #set_seed(random.randint(1,9999999))
  #texts = ["Windows 11 е най-якият процесор на AMD!","Intel пускат нова версия на операционната система AMD 11", "ЕИМ СВЯТ и юнаците срещу GPU-тата"]
  texts.append("Windows 11 е най-якият процесор на AMD!")
  texts.append("Intel пускат нова версия на операционната система AMD 11")
  texts.append("ЕИМ СВЯТ и юнаците срещу GPU-тата")
  #3-7-2021 prostotiya
  #texts = ["Да бъдеш или да не бъдеш - попита проститутката? Това е въпросът, задник! - отговори Терминаторът - застреля БМВ-то с пушката си и колата с Джон Конър се преобърна. Тогава момичето засмука хуя му, докато той не свърши в устата и тя рече: О, толкова ти е голям!"]
  
  b = beams
  top_k = 40
  sd = 0
  for text in texts:
    f.write(str(n) + ":" + ", BEAMS = "+str(b)+" seed: " + str(sd) + "t: " + str(t) + ", top_k: " + str(top_k) + "\r\n")
    #set_seed(random.randint(1,9999999))
    sd = random.randint(1,9999999)
    set_seed(sd)        
    gn = gen_return(tokenizer, length=length, beams=b, t=t, ngram=2, seq=1, top_k=top_k, text=text) #texts[n])
    for nm, item in enumerate(gn):
      output = "\r\n" + str(nm) + "\r\n" + tokenizer.decode(item)      
      nm+=1
    f.write(output)
    f.write("\r\n") 
    n+=1    
  f.close()  
  
#params = [0 = text, 1 = length, 2 = beams, 3 = temperature, 4 = ngram, 5 = top_k]
#or text - separated?
#def advancedSingleRet(tokenizer, text, params) --> str : 
def advancedSingleRet(tokenizer, model, params) -> str : 
  text, sd, length, beams, t, ngram, top_k = params
  seq = 1 #one only for now - possible future many and select from them etc. but too slow
  now = datetime.now() 
  #length = 200#125 #250 #250 #400 #150 #200  
  date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
  #f = open("gen.txt", "wt", encoding='utf-8') #date etc.
  fname = "gen-" +  date_time + ".txt"
  f = open(fname, "wt", encoding='utf-8') #date etc.  
  #beams = 8 #16 #8 #16 # 8 #16  
  #t = 1 #3 #1.2 #not 1.5 #.2 #until 24.6. incl. was 1.2 then a run ~ 20:00 25.6 =1; now 1.5
  #n = 0
  #ngram = 2 #2 #3 - not good # was 2 until 25.6 20:hh
  f.write("gn = gen_return(tokenizer, " + "length="+str(length)+" , beams= " + str(beams) + ", t=" +str(t) + ", ngram=" + str(ngram))
  print("Generating...")  
  input_ids = tokenizer.encode(text, return_tensors='tf')
  # getting out output
  beam_output = model.generate(
    input_ids,
    max_length = length, #150,
    num_beams = beams, #16,
    temperature = t, #1.0,
    no_repeat_ngram_size=ngram, #2,
    num_return_sequences=seq, #1
    top_k = top_k
  ) 
  #ret = []  
  #for i in range(0,seq):
  #  print(tokenizer.decode(beam_output[i]))    
  #  ret.append(beam_output[i])
  #return beam_output[0] #this is encoded! 
  dec = tokenizer.decode(beam_output[0]) #beam_output[0] #this is encoded!
  f.write(dec); #includes the input -- sometimes would be excluded in sequential generation
  f.close()
  return dec #tokenizer.decode(beam_output[0]) #beam_output[0] #this is encoded!
  

def advancedCycle(tokenizer, feed=None):
  pass
  
def genMany():
  text = "Той каза на жена си, че много я обича и винаги след нея ще тича, а тя се разсмя: Иване, купи ми кола или ще си намеря друг!"
  #advanced(text)
  set_seed(345)
  advancedMany(tokenizer, None)  
  

simpleEnd = ' '
def endSentence():
  if simpleEnd != None: return simpleEnd
  ends = ['.','!',';','?','...','?!']
  r = random.randint(1,len(ends)-1)
  return ends[r]
def endSentenceGeneration(text):
  return ' ' #more complex, ensure finishing previous burst properly
  #return '.' #more complex, ensure finishing previous burst properly
  
def genRecurse(): #28.6.2021
    '''
    0 -- characters, names
    1 -- locations
    2 -- topics
    3 -- specifics ...
    '''
    feed = []
    
    feed.append([["Емил"], ["Ада"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"]])
    feed.append([["София"], ["България"], ["Париж"], ["Франция"]])
    feed.append([["отивам"], ["пристигам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["телевизора"], ["колата"], ["ракия"], ["москвич"], ["летище"], ["самолет"], ["хляб"], ["нива"]])    
    
    ln = 80
    ln_step = 50 #80
    total = ''
    ftotal = open(r"e:\gpt\total.txt", "wt", encoding='utf-8')
    print(type(feed[0][0]),type(feed[1][0]),type(feed[2][0]),type(feed[3][0]))
    print(type(feed[0][0][0]),type(feed[1][0]),type(feed[2][0]),type(feed[3][0]))
    mix = feed[0][0][0] + " " + feed[1][0][0] + " " + feed[2][0][0] + " " + feed[3][0][0] + endSentence()
    sd = set_seed(random.randint(1,9999))
    par = (mix, sd, ln, 16, 1.2, 2, 40)
    lenmix = len(mix)
        
    #LOAD
    #'''
    model_dir = r"Z:\gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
    model = TFGPT2LMHeadModel.from_pretrained(model_dir)
    #'''

    ret = advancedSingleRet(tokenizer, model, par)
    print(ret)
    fn = ret.find(mix) #initial
    print("FN=",fn)
    if fn > -1:
               trim = ret[fn+len(mix):]
               print(fn+len(mix))
               ftotal.write("\r\n=======\r\n"+trim+"\r\n")
    else: trim = ret               
    total = trim + endSentenceGeneration(trim) #ret[len(mix):]  not+
    ftotal.write("\r\n"+ret[fn:fn+len(mix)] + "\r\n" + "\r\n" + mix + "\r\n=====\r\n")
    ftotal.write("\r\n"+total+"\r\n")
    
    #next = ret[len(mix):] #exclude the beginning already given
    next = total
    start = len(total)
    #to_add =". " + feed[0][1] + " на " + feed[1][1] + " тогава " + feed[2][1] + "когато " + feed[3][1]
    
    #ADD PRE-
    to_add = ". " + feed[0][1][0] + " за да " + feed[1][1][0] + " понеже " + feed[2][1][0] + " когато " + feed[3][1][0] + endSentence()
    #next = next + to_add
    next = to_add + next  #in the beginning in order to smooth the connection
    #new_start = len(to_add) + start #trim from here
    new_start = len(to_add) # + start #trim from here

    ln+=ln_step #should tokenize, check len etc...
    len_to_add = len(to_add)    
    sd = set_seed(random.randint(1,9999))
    
    input_ids = tokenizer.encode(next, return_tensors='tf')
    if len(input_ids) > ln: ln = len(input_ids)+lnstep
    
    par = (next, sd, ln, 16, 1.2, 2, 40)
    ret = advancedSingleRet(tokenizer,model, par)
    
    fn = ret.find(to_add) #, len(mix)) #this is not exactly - not precise; there are no repetitions anyway
    print("FN=",fn)    
    if fn > -1:
               trim = ret[fn+len(mix):]
               ftotal.write("\r\n=======\r\n"+trim+"\r\n")               
    else: trim = ret               
    #total += trim
    total +=ret[new_start:] + endSentenceGeneration(trim)
    print(fn)    
    
    #total += trim #ret[len(next):] 
    print(ret)
    print(total)
    ftotal.write("\r\n"+total+"\r\n")

    #THIRD    
    next = total #ret[len(mix):] #exclude the beginning already given
    pred_len = len(next)
    #to_add =". " + feed[0][1] + " на " + feed[1][1] + " тогава " + feed[2][1] + "когато " + feed[3][1]
    to_add = ". " + feed[0][2][0] + " към " + feed[1][2][0] + " защото " + feed[2][2][0] + " голям " + feed[3][2][0] + endSentence()       
    #next = next + to_add
    next = to_add + next
    #new_start = len(next) #to_add)
    new_start = len(to_add)
    ln+=ln_step
    len_to_add = len(to_add) 
    sd = set_seed(random.randint(1,9999))    
    
    input_ids = tokenizer.encode(next, return_tensors='tf')
    if len(input_ids) > ln: ln = len(input_ids)+lnstep
    par = (next, sd, ln, 16, 1.2, 2, 40)
    
    ret = advancedSingleRet(tokenizer,model, par)
    
    fn = ret.find(to_add, pred_len)
    print("FN=",fn)
    if fn > -1:
               trim = ret[fn+len(to_add):]
               ftotal.write("\r\n=======\r\n"+trim+"\r\n")
    else: trim = ret               
    #total += trim
    print(fn)
    
    total +=ret[new_start:] + endSentenceGeneration(trim)
    #total += trim #ret[len(next):] 
    #total += ret[len(next):] 
    print(ret)
    print(total)
    ftotal.write("\r\n"+total+"\r\n")

    ftotal.write(total)
    ftotal.close()

    #Generate in a batch etc. ... 
    #Interactive ... Keep model loaded etc. - takes a lot of time

    '''
      f.write("\r\nbeams = 8 etc. down to 1\r\n")
      gn = gen_return(tokenizer, length=150, beams=8, t=1.0, ngram=2, seq=1, text=None)
      f.write(''.join(tokenizer.decode(gn)))      
      f.write("\r\n")
      gn = gen_return(tokenizer, length=150, beams=4, t=1.0, ngram=2, seq=1, text=None)
      f.write(''.join(tokenizer.decode(gn)))      
      f.write("\r\n")
      gn = gen_return(tokenizer, length=150, beams=2, t=1.0, ngram=2, seq=1, text=None)
      f.write(''.join(tokenizer.decode(gn)))      
      f.write("\r\n")
      gn = gen_return(tokenizer, length=150, beams=1, t=1.0, ngram=2, seq=1, text=None)
      f.write(''.join(tokenizer.decode(gn)))    
      f.write("\r\n")
      f.close()
    '''
    


def genRecurseTwo():
    '''
    0 -- characters, names
    1 -- locations
    2 -- topics
    3 -- specifics ...
    '''
    feed = []
    '''
    feed.append([["Емил"], ["Ада"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"]])
    feed.append([["София"], ["България"], ["Париж"], ["Франция"]])
    feed.append([["отивам"], ["пристигам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["телевизора"], ["колата"], ["ракия"], ["москвич"], ["летище"], ["самолет"], ["хляб"], ["нива"]])    
  
    feed.append([["Емил"], ["Петя"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"]])
    feed.append([["Лондон"], ["България"], ["Париж"], ["Франция"]])
    feed.append([["идвам"], ["карам"],["пристигам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["професор"],["университет"], ["колата"], ["ракия"], ["москвич"], ["летище"], ["самолет"], ["хляб"], ["нива"]])
    '''
    '''
    feed.append([["Георги"], ["говори"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"]])
    feed.append([["Морето"], ["България"], ["Париж"], ["Франция"]])
    feed.append([["целувам"], ["красива"],["обичам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["леглото"],["града"], ["колата"], ["ракия"], ["москвич"], ["летище"], ["самолет"], ["хляб"], ["нива"]])
    '''
   
    feed.append([["жена"], ["процесор"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"]])
    #feed.append([["процесор"], ["България"], ["Париж"], ["Франция"], ["Италия"], ["Гърция"]])
    feed.append([["обичам"], ["България"], ["Париж"], ["Microsoft"], ["Италия"], ["Гърция"]])
    feed.append([["целувам"], ["красива"],["обичам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["леглото"],["града"], ["колата"], ["вода"], ["терминатор"], ["летище"], ["самолет"], ["хляб"], ["нива"]])
    
    #del feed
    #feed = []
    
    feed.append([["AMD"], ["процесор"], ["Intel"], ["секс"], ["ускорител"], ["Мария"]])
    #feed.append([["процесор"], ["България"], ["Париж"], ["Франция"], ["Италия"], ["Гърция"]])
    feed.append([["обичам"], ["България"], ["Париж"], ["Франция"], ["Италия"], ["1080"]])
    feed.append([["тест"], ["издание"],["обичам"], ["Microsoft"], ["съюз"], ["система"], ["видя"], ["целуна"], ["прегърна"]])
    feed.append([["летя"],["цена"], ["3080"], ["вода"], ["нов"], ["летище"], ["самолет"], ["хляб"], ["нива"]])
    
    feed = []
    '''
    feed.append([["жена"], ["процесор"], ["Иван"], ["Петър"], ["Ангел"], ["Мария"], ["хляб"], ["маса"]])
    #feed.append([["процесор"], ["България"], ["Париж"], ["Франция"], ["Италия"], ["Гърция"]])
    feed.append([["обичам"], ["България"], ["Париж"], ["Microsoft"], ["Италия"], ["Гърция"], ["часовник"], ["кацам"]])
    feed.append([["целувам"], ["красива"],["обичам"], ["заминавам"], ["кацам"], ["срещна"], ["видя"], ["целуна"], ["прегърна"], ["падам"]])
    feed.append([["леглото"],["града"], ["колата"], ["вода"], ["терминатор"], ["летище"], ["самолет"], ["общество"], ["нива"]])
    '''
    feed.append([["жена"], ["стая"], ["Тодор"], ["Терминатор"], ["Ангел"], ["целувам"], ["месо"], ["летя"]])
    #feed.append([["процесор"], ["България"], ["Париж"], ["Франция"], ["Италия"], ["Гърция"]])
    feed.append([["обичам"], ["лягам"], ["София"], ["Иван"], ["Италия"], ["гърди"], ["часовник"], ["кацам"]])
    feed.append([["целувам"], ["красива"],["обичам"], ["победа"], ["летище"], ["крака"], ["мъж"], ["целуна"], ["прегърна"], ["киборг"]])
    feed.append([["леглото"],["град"], ["колата"], ["тенис"], ["терминатор"], ["оргазъм"], ["самолет"], ["общество"], ["любов"]])


    #москвич
    print(len(feed[0]))
    #return
    #print(feed[0][0], feed[1][0], feed[2][0])
    # must be three redirections
    #return
    #1)
    ln = 120
    ln_step = 100 #100 #100 #80
    maxln = 200#220 #250 #180 less, was 180, to 130 - 3.7.2021
    bms = 8 #16
    total = ''
    now = datetime.now()
    date_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    fname = "gen-recurse-total-" +  date_time + ".txt"
    dir = ''    
    #ftotal = open(r"e:\gpt\totalTwo.txt", "wt", encoding='utf-8')
    ftotal = open(dir + fname, "wt", encoding='utf-8') #+ "total-" + ".txt"
    print(type(feed[0][0]),type(feed[1][0]),type(feed[2][0]),type(feed[3][0]))
    print(type(feed[0][0][0]),type(feed[1][0]),type(feed[2][0]),type(feed[3][0]))
    mix = feed[0][0][0] + " " + feed[1][0][0] + " " + feed[2][0][0] + " " + feed[3][0][0] + endSentence()
    sd = set_seed(random.randint(1,9999))
    par = (mix, sd, ln, bms, 1.2, 2, 40)
    lenmix = len(mix)
    last_context = 70 #80 #symbols from the previous stretch  #intermediate?
    #LOAD
    #'''
    #model_dir = r"Z:\gpt2"
    #model_dir = r"."
    ##tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
    ##model = TFGPT2LMHeadModel.from_pretrained(model_dir)
    ### USE the loaded model, #6-5-2023
    tokenizer = tokenizer2
    model = model2 
    #'''

    ret = advancedSingleRet(tokenizer, model, par)
    print(ret)
    fn = ret.find(mix) #initial
    print("FN=",fn)
    if fn > -1:
               trim = ret[fn+len(mix):]
               print(fn+len(mix))
               ftotal.write("\r\n=======================\r\n"+trim+"\r\n")
    else: trim = ret               
    total = trim + endSentenceGeneration(trim) #ret[len(mix):]  not+
    ftotal.write("\r\n"+ret[fn:fn+len(mix)] + "\r\n" + "\r\n" + mix + "\r\n=====\r\n")
    ftotal.write("\r\n"+total+"\r\n")
    
    #next = ret[len(mix):] #exclude the beginning already given
    next = total
    start = len(total)
    #to_add =". " + feed[0][1] + " на " + feed[1][1] + " тогава " + feed[2][1] + "когато " + feed[3][1]
    
    #last context + ... should split words etc.
    #ADD PRE-
    to_add = total[-last_context:] + " " + feed[0][1][0] + " за да " + feed[1][1][0] + " понеже " + feed[2][1][0] + " когато " + feed[3][1][0] + endSentence()
    #or POST?
    #to_add = feed[0][1][0] + " за да " + feed[1][1][0] + " понеже " + feed[2][1][0] + " когато " + feed[3][1][0] + endSentence() + total[-last_context:] + " "
    #next = next + to_add
    next = to_add # only the new words ... + next  #in the beginning in order to smooth the connection
    #new_start = len(to_add) + start #trim from here
    new_start = len(to_add) # + start #trim from here

    ln+=ln_step #should tokenize, check len etc...
    len_to_add = len(to_add)    
    sd = set_seed(random.randint(1,9999))
    temp = 2.0 #1.2
    par = (next, sd, ln, bms, temp, 2, 40)
    ret = advancedSingleRet(tokenizer,model, par)
    
    fn = ret.find(to_add) #, len(mix)) #this is not exactly - not precise; there are no repetitions anyway
    print("FN=",fn)    
    if fn > -1:
               trim = ret[fn+len(mix):]
               ftotal.write("\r\n=======\r\n"+trim+"\r\n")               
    else: trim = ret               
    #total += trim
    total +=ret[new_start:] + endSentenceGeneration(trim)
    print(fn)    
    
    #total += trim #ret[len(next):] 
    print(ret)
    print(total)
    ftotal.write("\r\n"+total+"\r\n")

    #THIRD    
    next = ret[-last_context:]  #ret[len(mix):] #exclude the beginning already given
    pred_len = len(next)
    #to_add =". " + feed[0][1] + " на " + feed[1][1] + " тогава " + feed[2][1] + "когато " + feed[3][1]
    to_add = ret[-last_context:] +  "  " +feed[0][2][0] + " към " + feed[1][2][0] + " защото " + feed[2][2][0] + " голям " + feed[3][2][0] + endSentence()
    #to_add = feed[0][2][0] + " към " + feed[1][2][0] + " защото " + feed[2][2][0] + " голям " + feed[3][2][0] + endSentence() + ret[-last_context:] + " "  #PRE new keywords
    
    #next = next + to_add
    next = to_add #+ next
    #new_start = len(next) #to_add)
    new_start = len(to_add)
    #ln+=ln_step
    len_to_add = len(to_add) 
    sd = set_seed(random.randint(1,9999))
    par = (next, sd, ln, bms, 1.0, 2, 40) #1.2, 2, 40)
    
    input_ids = tokenizer.encode(next, return_tensors='tf')
    if len(input_ids) > ln: ln = len(input_ids)+lnstep
    
    ret = advancedSingleRet(tokenizer,model, par)
    
    fn = ret.find(to_add, pred_len)
    print("FN=",fn)
    if fn > -1:
               trim = ret[fn+len(to_add):]
               ftotal.write("\r\n=======\r\n"+trim+"\r\n")
    else: trim = ret               
    #total += trim
    print(fn)
    
    total +=ret[new_start:] + endSentenceGeneration(trim)
    #total += trim #ret[len(next):] 
    #total += ret[len(next):] 
    print(ret)
    print(total)
    ftotal.write("\r\n"+total+"\r\n")

    choose = "към,защото,голям,малък,хубав,и,но,ако,с,без,да,където,когато,никога,ха-ха,не мога,мога,искам,не искам,къде,най-голям,стига,нея,аз,ти,той,тя,ние,вие,те"
    spl = choose.split(',')
    print(spl)
    space = " "
    for k in range(3,8): #5): #, len(feed[0])):     
      print("K=",k)    
      #random.choice(seq)¶
      #to_add = ret[-last_context:] + ". " + feed[0][k][0] + " към " + feed[1][k][0] + " защото " + feed[2][k][0] + " голям " + feed[3][k][0] + endSentence()      
      rch1 = space + random.choice(spl) + space  
      rch2 = space + random.choice(spl) + space
      rch3 = space + random.choice(spl) + space
      #rch4 = random.choice(seq)
      
      #step = ret[-last_context:] should split on whole words etc. ... #11-7-2021
      #to_add = ret[-last_context:] + ". " + feed[0][k][0] + " " + rch1 + " " + feed[1][k][0] + rch2 + feed[2][k][0] + rch3 + feed[3][k][0] + endSentence() 
      to_add = feed[0][k][0] + " " + rch1 + " " + feed[1][k][0] + rch2 + feed[2][k][0] + rch3 + feed[3][k][0] + endSentence() + ret[-last_context:] + " " # за да не прекъсва изреченията неестествено?
      
      ftotal.write("\r\n:TO_ADD=["+to_add+"]\r\n")
      next = to_add #+ next
      #new_start = len(next) #to_add)
      new_start = len(to_add)
      #ln+=ln_step
      len_to_add = len(to_add) 
      sd = set_seed(random.randint(1,9999))
      
      input_ids = tokenizer.encode(next, return_tensors='tf')
      if len(input_ids) > ln: ln = len(input_ids)+lnstep   
      else: ln = min(ln, maxln)
      print("LN=",ln)      
      par = (next, sd, ln, bms, 1.2, 2, 40)    
      ret = advancedSingleRet(tokenizer,model, par)
   
      fn = ret.find(to_add, pred_len)
      print("FN=",fn)
      if fn > -1:
                 trim = ret[fn+len(to_add):]
                 ftotal.write("\r\n=======\r\n"+trim+"\r\n")
      else: trim = ret               
      #total += trim
      print(fn)
    
      total +=ret[new_start:] + endSentenceGeneration(trim)
      #total += trim #ret[len(next):] 
      #total += ret[len(next):] 
      print(ret)
      print(total)
      ftotal.write("\r\n"+total+"\r\n")
      ftotal.write(total)      
 

    #Generate in a batch etc. ... 
    #Interactive ... Keep model loaded etc. - takes a lot of time  
    
#genRecurse()
#genRecurseTwo()
####if tokenizer == None: tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
####if model == None: model = TFGPT2LMHeadModel.from_pretrained(model_dir)

#genRecurseTwo()
#advancedMany(tokenizer)
#genRecurseTwo()
#advancedMany(tokenizer2)
model = model2
#generate(tokenizer2) #OK! 6-5-2023
#advancedMany(tokenizer2)
genRecurseTwo()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


8
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
<class 'str'> <class 'list'> <class 'list'> <class 'list'>
Generating...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


жена обичам целувам леглото  — казва баща ми и майка ми започва да плаче.
Прибирам се вкъщи след работа, докато мама помага на баща си. Майка ми разказва, че когато съм бил бебе, влязла в кухнята и заварила баба ми да ме кърми. Оттогава отношенията между двете жени съвсем не вървят. Точно сега не ми се удаде да живея в къща, близо до дома на баба си, която гледаше към къщата. Не знам защо, но този път ми раздрънка. Цялата къща бе пълна с деца, децата също като нея. Аз я харесвах и още ми беше мъчно за нея;
FN= 0
28
Generating...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


...

FN= -1
-1
Ангел  аз  Италия вие летище тя терминатор лед мен. Лягам на една от лобито вече не продумва. Не е ядосан, казва, 
Пиколо тук, не е алчен…
Ще минем навън, ще видиш!

Спусна се по стръмната рътлина на шосето. Пред нея течеше грамада вършини. Наоколо не се виждаше жива душа, но тя бе умряла и той не можеше да й се нарадва.
— Хайде, стига! — заповяда й той, а в същото време я прегърна и се загледа към покритата с гъста гора, на която се белееше равна и равна пътека. Тук-там се виждаха само бледосивите тънки силуети на „чашите“. Никой не разбра кога пак са се измъкнали. Другите ги настигнаха. Химикът хвърли на земята скатания брезент на палатката и потегли заедно с Янка, който летеше. Изведнъж той забеляза, че е в гората, дето се намираше нещо гигантско, и
 — казва баща ми и майка ми започва да плаче.
Прибирам се вкъщи след работа, докато мама помага на баща си. Майка ми разказва, че когато съм бил бебе, влязла в кухнята и заварила баба ми да ме кърми. Оттогава отношенията между 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


FN= -1
-1
целувам  хубав  гърди къде крака малък оргазъм веднъж той забеляза, че е в гората, дето се намираше нещо гигантско, и  по-нататък не съм го виждал. Или може би съм и аз като него.
И изведнъж се досетих. Видях в далечината кафявите скали на планината, бялата камбанария на селската църква и на самотната планина. Близо до нея се издигаха високи планини, все още покрити със сняг. Погледнах назад и видях трите коли, които пълзяха, разделени от облаци прах. Минахме покрай дълга върволица натоварени мулета. Водачите им, с червени фесове, вървяха отстрана край животните. Това бяха берсалиери. Един въоръжен с пушка в ръка им извика:
— Гери, ела при мене!
Северин хвана здравата револвера, но не можа да се стърпи и да види кой е пръв, като вече приближаваше земята. Силите му ръце бяха вързани и краката, защото краката не бяха мокри от ръжени, а краката му
 — казва баща ми и майка ми започва да плаче.
Прибирам се вкъщи след работа, докато мама помага на баща си. Майка ми разказва, че ког

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50250 for open-end generation.


FN= -1
-1
месо  той  часовник защото мъж те самолет ързани и краката, защото краката не бяха мокри от ръжени, а краката му  се бяха вкопчили в един ъгъл на друг в празничен ден. Пък и отговорът не беше толкова прост, колкото ми се стори през оня ден в кабинета на доктор Топалов. И аз изобщо нямаше да спомена тоя разговор, ако не се бяхме случили по-късно толкова важни мисли, които за пръв път са преценили. Във всеки случай не на себе си, но все пак имаше нещо подозрително.
— А къде се намираме? — запита Яна, навярно прикована отново в някаква кръчма, която намери на пода до самия под и едва тогава си направи труда да помоли турчина. — Добре, ще ви кажа. Но затуй пък…
Витан Велев като че ли дори се смали в ръцете му. Той се поусми за миг, после някак се спря, като откри, че не може да се обърне, и промълви:

 — казва баща ми и майка ми започва да плаче.
Прибирам се вкъщи след работа, докато мама помага на баща си. Майка ми разказва, че когато съм бил бебе, влязла в кухнята и заварила ба

In [28]:
genRecurseTwo()

8
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
<class 'str'> <class 'list'> <class 'list'> <class 'list'>


HFValidationError: ignored

In [13]:
print(bytes(tokenizer2.decode(out[0])))

TypeError: ignored

In [21]:
help (model2.generate)


Help on method generate in module transformers.generation.tf_utils:

generate(inputs: Optional[tensorflow.python.framework.ops.Tensor] = None, generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None, logits_processor: Optional[transformers.generation.tf_logits_process.TFLogitsProcessorList] = None, seed=None, **kwargs) -> Union[transformers.generation.tf_utils.TFGreedySearchEncoderDecoderOutput, transformers.generation.tf_utils.TFGreedySearchDecoderOnlyOutput, transformers.generation.tf_utils.TFSampleEncoderDecoderOutput, transformers.generation.tf_utils.TFSampleDecoderOnlyOutput, transformers.generation.tf_utils.TFBeamSearchEncoderDecoderOutput, transformers.generation.tf_utils.TFBeamSearchDecoderOnlyOutput, transformers.generation.tf_utils.TFBeamSampleEncoderDecoderOutput, transformers.generation.tf_utils.TFBeamSampleDecoderOnlyOutput, transformers.generation.tf_utils.TFContrastiveSearchEncoderDecoderOutput, transformers.generation.tf_utils.TF

# Нова секция

In [ ]:
for i in paths: print(i)
!du -sh /content/ab/2
print(len(paths))

/content/ab/2/Ljubomir_Nikolov_-_Konan_i_igrata_na_bogovete_-_5334-b.txt
/content/ab/2/ivo_hristov_svruh-obobshteniya_6-2021.txt
/content/ab/2/Stephen-King_-_Bjagashtijat_chovek_-_1257-b.txt
/content/ab/2/alyosha2.txt
/content/ab/2/Hristo Botev - V - . Revoljutsija v Krajovo - 3682.txt
/content/ab/2/Arthur-Hailey_-_Letishte_-_32-b.txt
/content/ab/2/yunaka-i-taz.txt
/content/ab/2/George-Orwell_-_1984_-_53-b.txt
/content/ab/2/tupanasov.txt
/content/ab/2/Hristo Botev - V - . Tova vi chaka! - 3668.txt
/content/ab/2/Kathy-Lette_-_Ako_vibratorite_imaha_klepachi_-_6297-b.txt
/content/ab/2/Giacomo-Casanova_-_Prikljuchenijata_na_Kazanova_-_Tom_1_-_6751-b.txt
/content/ab/2/EIM-smetach-do-br-6-moje-da-e-nepulno-bg.txt
/content/ab/2/terminator-i-herkules.txt
/content/ab/2/Hristo Botev - V - . Epitafii - 3679.txt
/content/ab/2/Hristo Botev -  - . Do moeto pyrvo libe - 3231.txt
/content/ab/2/Ivan_Hadzhijski_-_Optimistichna_teorija_za_nashija_narod_-_7943-b.txt
/content/ab/2/kakvomu_notes.txt
/conten

In [ ]:
!ls /content/ab/

 135-godini.txt
 Ada_12-10_E-potursij-i-po-novi-vtoro-chislo-19-7-2020-Arial-p189_Arial16-Big-Font.txt
 Alan-Williams_-_Arhivyt_na_Berija_-_Kniga_pyrva_-_10179-b.txt
 Aldous-Huxley_-_Prekrasnijat_nov_svjat_-_625-b.txt
'Aleko Konstantinov -  - . Do Chikago i nazad - 3705.txt'
 alyosha1.txt
 alyosha2.txt
 AMD-Intel.txt
 Andre-Maurois_-_Bajron_-_3615-b.txt
 Angel_Karalijchev_-_Bylgarski_narodni_prikazki_-_652-b.txt
 baladi.txt
 Balzak-chast.txt
 belezhki-istinata.txt
 bivalica.txt
 Bruder-Grimm_-_Prikazki_-_648-b.txt
 Bylgarski_narodni_vylshebni_prikazki_-_5768-b.txt
 Charles-Perrault_-_Chervenata_shapchitsa_i_drugi_prikazki_-_10184-b.txt
 chlen.txt
 comment-nerds-2-nerds-128-clean1.txt
 darenie.txt
 democratius_prilozhenie_drugi.txt
 democratius_stamboliyski.txt
 doktoratut.txt
 Douglas-Adams_-_Pytevoditel_na_galakticheskija_stopadzhija_-_961-b.txt
 dve-narechiya.txt
 DZBE_ponyatiya_7-2020_eim-33.txt
 EIM-smetach-do-br-6-moje-da-e-nepulno-bg.txt
 Elin_Pelin_-_Sychinenija_v_shest_toma_tom